In [6]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:22pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.text_cell_render.rendered_html{font-size:15pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:18pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:18pt;padding:5px;}
table.dataframe{font-size:18px;}
</style>
"""))

In [ ]:
#  요기요 웹크롤링

In [ ]:
# 네이버/구글 이미지 검색 수집 (이미지, 날짜, 장소 등 가능한지)

In [2]:
from dotenv import load_dotenv
import os
import sys
import urllib.request
import urllib.parse
import pandas as pd
import json
load_dotenv()

True

` 방법 
1.네이버 이미지 관련 크롤링 
2.이미지 url, 게시일자 추출
3.제목,본문에서 <가게명,위치,가격 등 패턴 정규식 추출>
4.이미지다운로드
5.구조화된 데이터로 정리 (csv)
`

In [28]:
# 이미지 검색 
# 환경변수 키 불러오기
client_id = os.getenv('Client_ID')
client_secret = os.getenv('Client_Secret')

# 검색어 인코딩
encText = urllib.parse.quote('서울 짜장면')
url = f"https://openapi.naver.com/v1/search/blog?query={encText}&display=100" # JSON 결과 

# Request 객체 생성 + 헤더 추가 
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id", client_id)
request.add_header("X-Naver-Client-Secret", client_secret)

# 응답 받기
with urllib.request.urlopen(request) as response:
    result = response.read().decode('utf-8')

# JSON 파싱
data = json.loads(result)
items = data['items']

# 리스트로 정리 
items_list = []
for item in items:
    items_list.append([
        item.get('title').replace('<b>','').replace('</b>',''),
        item.get('link'),
        item.get('description').replace('<b>','').replace('</b>',''),
        item.get('postdate')
    ])
    
# DataFrame 생성
seoul_black_noodle_100 = pd.DataFrame(items_list,columns=['title','link','postdate','description'])
soeul_black_noodle_100

# 데이터 프레임 저장
df.to_csv('data/test1_seoul_black_noodle_100.csv',encoding='utf-8-sig' ,index=False)

# 데이터 추출
# import json
# items = response.json()['items']

In [23]:
seoul_black_noodle_100.iloc[:,2:3]

,postdate
0,업체에 서비스를 제공받아 직접 방문하여 작성한 리뷰입니다 맛돌이들이라면 모두 아는 ...
1,잘 먹었습니다. 또 오겠습니다. 주관적 평점 5 #팔공 #중화요리팔공 #관악구맛집 ...
2,#광화문맛집 #서울광화문맛집 #광화문역맛집 #광화문주변맛집 #광화문역점심맛집 #경복...
3,"서촌 맛집 추천, 서울 경복궁 서촌마을 짜장면 중식 혼밥 맛있는 식당 서울 서촌 마..."
4,서울 짜장면 맛집! 서울 동대문 창신동 맛집! 전통 수타면 방식의 동대문 일품손짜장...
...,...
95,#짜장면 #간짜장 #짜장면맛집 #서울짜장면맛집 #중식당 #블랙데이 #짜장면데이 #홍...
96,"중화요리 생각날 때 맛있게 먹고 싶을 때 최근에 먹어본 짜장면, 짬뽕, 탕수육 서울..."
97,"채끝살 김치짜장면 서울시스터즈 김치짜장면, 채끝살, 트러플오일,양파 채끝살 김치짜장..."
98,구매는 여기서 https://www.rodemfoodshop.com/goods/go...


네이버 지도에서 가져오는 방법

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import re

In [4]:
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
# chrome_options.add_argument("--headless")  # 창 안 띄우고 싶다면 주석 해제

# ▶ 크롬 드라이버 실행
driver = webdriver.Chrome(options=chrome_options)

# ▶ 네이버 지도 서울 음식점 검색 URL
driver.get("https://map.naver.com/p/search/서울 신림동 음식점")
time.sleep(5)

# ▶ 검색 결과 iframe 진입
driver.switch_to.frame("searchIframe")
time.sleep(2)

# place_ids = set()

# # ▶ 페이지 반복 크롤링
# for page in range(1, 20):  # 🔁 필요시 더 늘리기 (예: 100 페이지까지)
#     print(f"[INFO] 페이지 {page} 처리 중...")
#     time.sleep(2)

#     items = driver.find_elements(By.CSS_SELECTOR, 'a.place_bluelink')

#     for item in items:
#         href = item.get_attribute('href')
#         match = re.search(r'place/(\d+)', href)
#         if match:
#             place_ids.add(match.group(1))

#     # ▶ 다음 페이지 버튼 클릭
#     try:
#         next_btn = driver.find_element(By.CSS_SELECTOR, 'a.pagination.next')
#         if 'disabled' in next_btn.get_attribute('class'):
#             break
#         next_btn.click()
#     except:
#         print("[INFO] 다음 페이지 없음 또는 에러.")
#         break

# # ▶ 드라이버 종료
# driver.quit()

# # ▶ 결과 저장
# with open("seoul_place_ids.txt", "w") as f:
#     for pid in sorted(place_ids):
#         f.write(f"{pid}\n")

# print(f"[DONE] 총 수집된 placeId: {len(place_ids)}개")

# 네이버 지도에서 사진을 통해 업체별 사진 받아는 방법인데 아래꺼 좀 더 정리하면 될듯... 

In [8]:
driver = webdriver.Chrome()
place_url = "https://naver.me/5vcHMukK"  # 예: 스타벅스 종로점
driver.get(place_url)
time.sleep(5)

# 2. iframe 내부 접근
driver.switch_to.frame(driver.find_element(By.CSS_SELECTOR, "iframe#entryIframe"))
time.sleep(2)

# 3. "사진" 탭 클릭
try:
    photo_tab = driver.find_element(By.CSS_SELECTOR, 'a[href*="photo"]')
    photo_tab.click()
    time.sleep(3)
except Exception as e:
    print("사진 탭 클릭 실패:", e)

# 4. 스크롤해서 이미지 로딩 더 하기
for _ in range(10):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

# 5. 이미지 태그 수집
img_elements = driver.find_elements(By.CSS_SELECTOR, "img")  # 전체 이미지 태그 수집
os.makedirs("naver_images/starbucks_photos", exist_ok=True)

# 6. 이미지 저장
for i, img in enumerate(img_elements):
    try:
        src = img.get_attribute("src")
        if src and src.startswith("https") and "place" in src:
            urllib.request.urlretrieve(src, f"naver_images/starbucks_photos/img_{i}.jpg")
    except Exception as e:
        print(f"이미지 저장 실패 {i}: {e}")

driver.quit()
print("완료!")

완료!


# 신림동 음식점 정보 추출기 (이름,주소, placeld)


In [4]:
def switch_left():
    ############## iframe으로 왼쪽 포커스 맞추기 ##############
    driver.switch_to.parent_frame()
    iframe = driver.find_element(By.XPATH,'//*[@id="searchIframe"]')
    driver.switch_to.frame(iframe)
    
def switch_right():
    ############## iframe으로 오른쪽 포커스 맞추기 ##############
    driver.switch_to.parent_frame()
    iframe = driver.find_element(By.XPATH,'//*[@id="entryIframe"]')
    driver.switch_to.frame(iframe)

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
 
from time import sleep
import random
import re
 
from selenium import webdriver
import sys
 
from selenium.webdriver.chrome.options import Options
options = Options()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(options=options)

# 1. 네이버 지도 검색 페이지 열기 (신림동 음식점)
URL = "https://map.naver.com/p/search/신림동 햄버거"
driver.get(url=URL)

# 2. 검색 결과 iframe 진입 
driver.switch_to.frame("searchIframe")

# 2. 페이지 숫자 체크
while(True):
    switch_left()
 
    # 페이지 숫자를 초기에 체크 [ True / False ]
    # 이건 페이지 넘어갈때마다 계속 확인해줘야 함 (페이지 새로 로드 될때마다 버튼 상태 값이 바뀜)
    next_page = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div[3]/div[2]/a[7]').get_attribute('aria-disabled')
    
    if(next_page == 'true'):
        break

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="app-root"]/div/div[3]/div[2]/a[7]"}
  (Session info: chrome=138.0.7204.97); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x7ff73c2a6f95+76917]
	GetHandleVerifier [0x0x7ff73c2a6ff0+77008]
	(No symbol) [0x0x7ff73c059dea]
	(No symbol) [0x0x7ff73c0b0256]
	(No symbol) [0x0x7ff73c0b050c]
	(No symbol) [0x0x7ff73c103887]
	(No symbol) [0x0x7ff73c0d84af]
	(No symbol) [0x0x7ff73c10065c]
	(No symbol) [0x0x7ff73c0d8243]
	(No symbol) [0x0x7ff73c0a1431]
	(No symbol) [0x0x7ff73c0a21c3]
	GetHandleVerifier [0x0x7ff73c57d2cd+3051437]
	GetHandleVerifier [0x0x7ff73c577923+3028483]
	GetHandleVerifier [0x0x7ff73c5958bd+3151261]
	GetHandleVerifier [0x0x7ff73c2c185e+185662]
	GetHandleVerifier [0x0x7ff73c2c971f+218111]
	GetHandleVerifier [0x0x7ff73c2afb14+112628]
	Ge

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
 
from time import sleep
import random
import re
 
from selenium import webdriver
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

# 1. 네이버 지도 열기
URL = "https://map.naver.com/p/search/신림동 음식점"
driver.get(URL)
time.sleep(5)

# 2. 검색 결과 iframe 진입
driver.switch_to.frame("searchIframe")
time.sleep(2)

# 3. 다음 페이지 존재 여부 확인 반복
while True:
    print("[INFO] 현재 페이지 처리 중...")

    # TODO: 여기서 음식점 정보 수집 가능

    try:
        next_btn = driver.find_element(By.CSS_SELECTOR, "a.pagination.next")
        aria_disabled = next_btn.get_attribute("aria-disabled")
        
        if aria_disabled == "true":
            print("[DONE] 마지막 페이지 도달")
            break

        next_btn.click()
        time.sleep(3)

    except Exception as e:
        print("[ERROR] 다음 페이지 버튼 찾기 실패:", e)
        break

driver.quit()

NoSuchFrameException: Message: searchIframe


In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

from time import sleep
import random
import re

from selenium import webdriver
import sys

options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Mobile Safari/537.36')
options.add_argument('window-size=1380,900')
driver = webdriver.Chrome(options=options)

# 대기 시간
driver.implicitly_wait(time_to_wait=3)

# 반복 종료 조건
loop = True

URL = 'https://map.naver.com/p/search/%EC%9D%B8%ED%95%98%EB%8C%80%20%EC%9D%8C%EC%8B%9D%EC%A0%90?c=15.23,0,0,0,dh'
driver.get(url=URL)

while(True):
    switch_left()

    # 페이지 숫자를 초기에 체크 [ True / False ]
    # 이건 페이지 넘어갈때마다 계속 확인해줘야 함 (페이지 새로 로드 될때마다 버튼 상태 값이 바뀜)
    next_page = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div[3]/div[2]/a[7]').get_attribute('aria-disabled')
    
    if(next_page == 'true'):
        break

    ############## 맨 밑까지 스크롤 ##############
    scrollable_element = driver.find_element(By.CLASS_NAME, "Ryr1F")

    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)

    while True:
        # 요소 내에서 아래로 600px 스크롤
        driver.execute_script("arguments[0].scrollTop += 600;", scrollable_element)

        # 페이지 로드를 기다림
        sleep(1)  # 동적 콘텐츠 로드 시간에 따라 조절

        # 새 높이 계산
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)

        # 스크롤이 더 이상 늘어나지 않으면 루프 종료
        if new_height == last_height:
            break

        last_height = new_height


    ############## 현재 page number 가져오기 - 1 페이지 ##############

    page_no = driver.find_element(By.XPATH,'//a[contains(@class, "mBN2s qxokY")]').text

    # 현재 페이지에 등록된 모든 가게 조회
    # 첫페이지 광고 2개 때문에 첫페이지는 앞 2개를 빼야함
    if(page_no == '1'):
        elemets = driver.find_elements(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]//li')[2:]
    else:
        elemets = driver.find_elements(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]//li')

    print('현재 ' + '\033[95m' + str(page_no) + '\033[0m' + ' 페이지 / '+ '총 ' + '\033[95m' + str(len(elemets)) + '\033[0m' + '개의 가게를 찾았습니다.\n')

    for index, e in enumerate(elemets, start=1):
        final_element = e.find_element(By.CLASS_NAME,'CHC5F').find_element(By.XPATH, ".//a/div/div/span")

        print(str(index) + ". " + final_element.text)

    print(Colors.RED + "-"*50 + Colors.RESET)

    switch_left()

    sleep(2)

    for index, e in enumerate(elemets, start=1):
        store_name = '' # 가게 이름
        category = '' # 카테고리
        new_open = '' # 새로 오픈
        
        rating = 0.0 # 평점
        visited_review = 0 # 방문자 리뷰
        blog_review = 0 # 블로그 리뷰
        store_id = '' # 가게 고유 번호
        
        address = '' # 가게 주소
        business_hours = [] # 영업 시간
        phone_num = '' # 전화번호

        switch_left()


        # 순서대로 값을 하나씩 클릭
        e.find_element(By.CLASS_NAME,'CHC5F').find_element(By.XPATH, ".//a/div/div/span").click()

        sleep(2)

        switch_right()

        ################### 여기부터 크롤링 시작 ##################
        title = driver.find_element(By.XPATH,'//div[@class="zD5Nm undefined"]')
        store_info = title.find_elements(By.XPATH,'//div[@class="YouOG DZucB"]/div/span')


        # 가게 이름
        store_name = title.find_element(By.XPATH,'.//div[1]/div[1]/span[1]').text

        # 카테고리
        category = title.find_element(By.XPATH,'.//div[1]/div[1]/span[2]').text

        if(len(store_info) > 2):
            # 새로 오픈
            new_open = title.find_element(By.XPATH,'.//div[1]/div[1]/span[3]').text


        ###############################

        review = title.find_elements(By.XPATH,'.//div[2]/span')

        # 인덱스 변수 값
        _index = 1

        # 리뷰 ROW의 갯수가 3개 이상일 경우 [별점, 방문자 리뷰, 블로그 리뷰]
        if len(review) > 2:
            rating_xpath = f'.//div[2]/span[{_index}]'
            rating_element = title.find_element(By.XPATH, rating_xpath)
            rating = rating_element.text.replace("\n", " ")

            _index += 1

        try:
            # 방문자 리뷰
            visited_review = title.find_element(By.XPATH,f'.//div[2]/span[{_index}]/a').text


            # 인덱스를 다시 +1 증가 시킴
            _index += 1

            # 블로그 리뷰
            blog_review = title.find_element(By.XPATH,f'.//div[2]/span[{_index}]/a').text
        except:
            print(Colors.RED + '------------ 리뷰 부분 오류 ------------' + Colors.RESET)

        # 가게 id
        store_id = driver.find_element(By.XPATH,'//div[@class="flicking-camera"]/a').get_attribute('href').split('/')[4]







        # 가게 주소
        address = driver.find_element(By.XPATH,'//span[@class="LDgIH"]').text


        try:
            driver.find_element(By.XPATH,'//div[@class="y6tNq"]//span').click()

            # 영업 시간 더보기 버튼을 누르고 2초 반영시간 기다림
            sleep(2)


            parent_element = driver.find_element(By.XPATH,'//a[@class="gKP9i RMgN0"]')
            child_elements = parent_element.find_elements(By.XPATH, './*[@class="w9QyJ" or @class="w9QyJ undefined"]')

            for child in child_elements:
                # 각 자식 요소 내에서 클래스가 'A_cdD'인 span 요소 찾기
                span_elements = child.find_elements(By.XPATH, './/span[@class="A_cdD"]')

                # 찾은 span 요소들의 텍스트 출력
                for span in span_elements:
                    business_hours.append(span)
            
            # 가게 전화번호
            phone_num = driver.find_element(By.XPATH,'//span[@class="xlx7Q"]').text

        except:
            print(print(Colors.RED + '------------ 영업시간 / 전화번호 부분 오류 ------------' + Colors.RESET))
        
        print(Colors.BLUE + f'{index}. ' + str(store_name) + Colors.RESET + ' · ' + str(category) + Colors.RED + str(new_open) + Colors.RESET)
        print('평점 ' + Colors.RED + str(rating) + Colors.RESET + ' / ' + visited_review + ' · ' + blog_review)
        print(f'가게 고유 번호 -> {store_id}')
        print('가게 주소 ' + Colors.GREEN + str(address) + Colors.RESET)
        print(Colors.CYAN + '가게 영업 시간' + Colors.RESET)
        for i in business_hours:
            print(i.text)
            print('')
        print('가게 번호 ' + Colors.GREEN + phone_num + Colors.RESET)
        print(Colors.MAGENTA + "-"*50 + Colors.RESET)
    
    switch_left()
        
    # 페이지 다음 버튼이 활성화 상태일 경우 계속 진행
    if(next_page == 'false'):
        driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div[3]/div[2]/a[7]').click()
    # 아닐 경우 루프 정지
    else:
        loop = False

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="app-root"]/div/div[3]/div[2]/a[7]"}
  (Session info: chrome=138.0.7204.97); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x7ff68b906f95+76917]
	GetHandleVerifier [0x0x7ff68b906ff0+77008]
	(No symbol) [0x0x7ff68b6b9dea]
	(No symbol) [0x0x7ff68b710256]
	(No symbol) [0x0x7ff68b71050c]
	(No symbol) [0x0x7ff68b763887]
	(No symbol) [0x0x7ff68b7384af]
	(No symbol) [0x0x7ff68b76065c]
	(No symbol) [0x0x7ff68b738243]
	(No symbol) [0x0x7ff68b701431]
	(No symbol) [0x0x7ff68b7021c3]
	GetHandleVerifier [0x0x7ff68bbdd2cd+3051437]
	GetHandleVerifier [0x0x7ff68bbd7923+3028483]
	GetHandleVerifier [0x0x7ff68bbf58bd+3151261]
	GetHandleVerifier [0x0x7ff68b92185e+185662]
	GetHandleVerifier [0x0x7ff68b92971f+218111]
	GetHandleVerifier [0x0x7ff68b90fb14+112628]
	Ge

In [4]:
import json 
import time
import random
import re
import sys
import urllib.parse


from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options

##### 크롬 인터넷 설정
options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36')
options.add_argument('window-size=1380,900')
driver = webdriver.Chrome(options=options)

#####  대기시간 
driver.implicitly_wait(time_to_wait=3)


# 1. 네이버 지도 열기
keyword = "신림동 짜장면"
encoded_keyword = urllib.parse.quote(keyword)
URL = f"https://map.naver.com/p/search/{encoded_keyword}"

driver.get(URL)
time.sleep(5)

# 2. 검색창  iframe 진입
driver.switch_to.frame("searchIframe")
time.sleep(2)

# 검색창 찾기 + 검색어 입력
search_box = driver.find_element(By.CSS_SELECTOR, 'input[type="text"][ID="input_search1751588403392"]')
search_box.send_keys(keyword)
search_box.send_keys(Keys.ENTER)

time.sleep(5) # 검색 결과 기다리기 

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"input[type="text"][ID="input_search1751588403392"]"}
  (Session info: chrome=138.0.7204.97); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x7ff7cf8a6f95+76917]
	GetHandleVerifier [0x0x7ff7cf8a6ff0+77008]
	(No symbol) [0x0x7ff7cf659dea]
	(No symbol) [0x0x7ff7cf6b0256]
	(No symbol) [0x0x7ff7cf6b050c]
	(No symbol) [0x0x7ff7cf703887]
	(No symbol) [0x0x7ff7cf6d84af]
	(No symbol) [0x0x7ff7cf70065c]
	(No symbol) [0x0x7ff7cf6d8243]
	(No symbol) [0x0x7ff7cf6a1431]
	(No symbol) [0x0x7ff7cf6a21c3]
	GetHandleVerifier [0x0x7ff7cfb7d2cd+3051437]
	GetHandleVerifier [0x0x7ff7cfb77923+3028483]
	GetHandleVerifier [0x0x7ff7cfb958bd+3151261]
	GetHandleVerifier [0x0x7ff7cf8c185e+185662]
	GetHandleVerifier [0x0x7ff7cf8c971f+218111]
	GetHandleVerifier [0x0x7ff7cf8af

In [1]:
import json 
import time
import random
import re
import sys
import urllib.parse
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException

##### 크롬 인터넷 설정
options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36')
options.add_argument('window-size=1380,900')
driver = webdriver.Chrome(options=options)

##### 대기시간 설정
driver.implicitly_wait(10)
wait = WebDriverWait(driver, 15)

try:
    # 1. 네이버 지도 열기
    keyword = "신림동 짜장면"
    encoded_keyword = urllib.parse.quote(keyword)
    URL = f"https://map.naver.com/p/search/{encoded_keyword}"
    
    print(f"네이버 지도 접속 중: {URL}")
    driver.get(URL)
    time.sleep(5)
    
    # 2. 검색창 iframe 진입
    print("iframe 찾는 중...")
    
    # iframe 선택자들을 순서대로 시도
    iframe_selectors = [
        "iframe#searchIframe",
        "iframe[name='searchIframe']",
        "iframe[src*='search']",
        "iframe"
    ]
    
    iframe_found = False
    for selector in iframe_selectors:
        try:
            iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))
            driver.switch_to.frame(iframe)
            print(f"iframe 진입 성공: {selector}")
            iframe_found = True
            break
        except TimeoutException:
            print(f"iframe 선택자 실패: {selector}")
            continue
    
    if not iframe_found:
        print("iframe을 찾을 수 없음, 메인 페이지에서 검색창 찾기 시도...")
        driver.switch_to.default_content()
    
    time.sleep(2)
    
    # 3. 검색창 찾기
    print("검색창 찾는 중...")
    
    # 다양한 검색창 선택자들
    search_selectors = [
        "input[id*='input_search']",      # ID에 input_search가 포함된 요소
        "input.input_search",             # 클래스가 input_search인 요소
        "input[placeholder*='검색']",      # placeholder에 검색이 포함된 요소
        "input[type='text']",             # text 타입의 input
        "input[aria-label*='검색']",       # aria-label에 검색이 포함된 요소
        ".input_search",                  # 클래스 선택자
        "#input_search",                  # ID 선택자 (동적 ID 제외)
        "input"                           # 모든 input 태그
    ]
    
    search_box = None
    for selector in search_selectors:
        try:
            search_box = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, selector)))
            print(f"검색창 발견: {selector}")
            break
        except TimeoutException:
            print(f"선택자 실패: {selector}")
            continue
    
    # 만약 여전히 검색창을 찾지 못했다면, 모든 input 요소들을 확인
    if search_box is None:
        print("일반적인 선택자로 검색창을 찾지 못함. 모든 input 요소 확인 중...")
        try:
            inputs = driver.find_elements(By.TAG_NAME, "input")
            for i, input_elem in enumerate(inputs):
                try:
                    input_id = input_elem.get_attribute("id")
                    input_class = input_elem.get_attribute("class")
                    input_placeholder = input_elem.get_attribute("placeholder")
                    print(f"Input {i}: id='{input_id}', class='{input_class}', placeholder='{input_placeholder}'")
                    
                    # 검색과 관련된 속성이 있는지 확인
                    if (input_id and 'search' in input_id.lower()) or \
                       (input_class and 'search' in input_class.lower()) or \
                       (input_placeholder and '검색' in input_placeholder):
                        search_box = input_elem
                        print(f"검색창으로 추정되는 요소 발견: {input_elem}")
                        break
                except Exception as e:
                    print(f"Input {i} 속성 확인 중 오류: {e}")
                    continue
        except Exception as e:
            print(f"Input 요소들 확인 중 오류: {e}")
    
    if search_box is None:
        print("검색창을 찾을 수 없습니다.")
        print("현재 페이지 소스 (일부):")
        print(driver.page_source[:2000])
        raise Exception("검색창을 찾을 수 없음")
    
    # 4. 검색어 입력
    print(f"검색어 입력: {keyword}")
    try:
        # 검색창이 활성화될 때까지 대기
        wait.until(EC.element_to_be_clickable(search_box))
        
        # 기존 텍스트 제거
        search_box.clear()
        time.sleep(0.5)
        
        # 새로운 검색어 입력
        search_box.send_keys(keyword)
        time.sleep(1)
        
        # Enter 키 입력
        search_box.send_keys(Keys.ENTER)
        print("검색어 입력 및 엔터 완료")
        
    except Exception as e:
        print(f"검색어 입력 중 오류: {e}")
        # 대안: JavaScript로 값 설정
        try:
            driver.execute_script("arguments[0].value = arguments[1];", search_box, keyword)
            driver.execute_script("arguments[0].dispatchEvent(new Event('input', { bubbles: true }));", search_box)
            search_box.send_keys(Keys.ENTER)
            print("JavaScript로 검색어 입력 완료")
        except Exception as e2:
            print(f"JavaScript 입력도 실패: {e2}")
            raise
    
    print("검색 완료! 결과 대기 중...")
    time.sleep(5)
    
    # 5. 검색 결과 확인
    try:
        # 메인 프레임으로 돌아가기 (검색 결과는 보통 메인 프레임에 있음)
        driver.switch_to.default_content()
        
        # 검색 결과 요소들 찾기
        result_selectors = [
            "div[data-id]",
            ".place_bluelink",
            ".item_info",
            ".search_item",
            "li[data-index]"
        ]
        
        results = None
        for selector in result_selectors:
            try:
                results = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, selector)))
                print(f"검색 결과 {len(results)}개 발견 (선택자: {selector})")
                break
            except TimeoutException:
                print(f"결과 선택자 실패: {selector}")
                continue
        
        if results is None:
            print("검색 결과를 찾을 수 없음")
        else:
            print(f"총 {len(results)}개의 검색 결과 발견")
            
            # 첫 번째 결과의 정보 출력 (예시)
            if len(results) > 0:
                first_result = results[0]
                print(f"첫 번째 결과 텍스트: {first_result.text[:100]}...")
                
    except TimeoutException:
        print("검색 결과를 찾을 수 없음")
    
    print("크롤링 완료!")
    
except Exception as e:
    print(f"오류 발생: {e}")
    print(f"현재 URL: {driver.current_url}")
    
    # 디버깅을 위해 스크린샷 저장
    try:
        driver.save_screenshot("error_screenshot.png")
        print("오류 스크린샷 저장: error_screenshot.png")
    except:
        pass
    
finally:
    # 브라우저 종료
    input("엔터를 누르면 브라우저가 종료됩니다...")
    driver.quit()

네이버 지도 접속 중: https://map.naver.com/p/search/%EC%8B%A0%EB%A6%BC%EB%8F%99%20%EC%A7%9C%EC%9E%A5%EB%A9%B4
iframe 찾는 중...
iframe 진입 성공: iframe#searchIframe
검색창 찾는 중...
선택자 실패: input[id*='input_search']
선택자 실패: input.input_search
선택자 실패: input[placeholder*='검색']
선택자 실패: input[type='text']
선택자 실패: input[aria-label*='검색']
선택자 실패: .input_search
선택자 실패: #input_search
선택자 실패: input
일반적인 선택자로 검색창을 찾지 못함. 모든 input 요소 확인 중...
검색창을 찾을 수 없습니다.
현재 페이지 소스 (일부):
<html lang="ko"><head>
        <title>네이버 플레이스</title>
        <meta charset="utf-8">
        <meta httpequiv="x-ua-compatible" content="ie=edge">
        <meta id="_viewport" name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no, shrink-to-fit=no, viewport-fit=cover">
        <meta name="format-detection" content="telephone=no">
        <link rel="icon" type="image/png" href="https://g-place.pstatic.net/assets/shared/images/favicon-72.png">
        <link rel="icon" type="image/png"

In [2]:
import json 
import time
import os
import urllib.parse
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException

def create_directory(path):
    """디렉토리 생성"""
    if not os.path.exists(path):
        os.makedirs(path)

def download_image(url, filename):
    """이미지 다운로드"""
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        with open(filename, 'wb') as f:
            f.write(response.content)
        return True
    except Exception as e:
        print(f"이미지 다운로드 실패: {e}")
        return False

def extract_restaurant_info(driver, wait):
    """음식점 정보 추출"""
    restaurants = []
    
    try:
        # 검색 결과 컨테이너 찾기
        result_selectors = [
            "div._pcmap_list_scroll_container",
            "div[data-grid-container]",
            "div.place_section_content",
            "div.search_list_area",
            "ul.search_list"
        ]
        
        container = None
        for selector in result_selectors:
            try:
                container = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))
                print(f"검색 결과 컨테이너 발견: {selector}")
                break
            except TimeoutException:
                continue
        
        if container is None:
            print("검색 결과 컨테이너를 찾을 수 없음")
            return restaurants
        
        # 개별 음식점 요소들 찾기
        restaurant_selectors = [
            "div[data-id]",
            "li[data-index]",
            "div.place_bluelink",
            "div.search_item",
            "div._list_item"
        ]
        
        restaurant_elements = []
        for selector in restaurant_selectors:
            try:
                elements = container.find_elements(By.CSS_SELECTOR, selector)
                if elements:
                    restaurant_elements = elements
                    print(f"음식점 요소 {len(elements)}개 발견: {selector}")
                    break
            except:
                continue
        
        if not restaurant_elements:
            print("음식점 요소를 찾을 수 없음")
            return restaurants
        
        # 각 음식점 정보 추출
        for i, element in enumerate(restaurant_elements[:10]):  # 최대 10개만 처리
            try:
                restaurant_info = {}
                
                # 음식점 이름
                name_selectors = [
                    "span.place_bluelink",
                    "span._title",
                    "div.place_name",
                    "strong",
                    "a.place_bluelink"
                ]
                
                name = None
                for selector in name_selectors:
                    try:
                        name_elem = element.find_element(By.CSS_SELECTOR, selector)
                        name = name_elem.text.strip()
                        if name:
                            break
                    except:
                        continue
                
                if not name:
                    continue
                
                restaurant_info['name'] = name
                print(f"\n{i+1}. {name}")
                
                # 별점
                rating_selectors = [
                    "span.place_rating_star",
                    "span._rating",
                    "em.place_rating",
                    "span[class*='rating']"
                ]
                
                rating = None
                for selector in rating_selectors:
                    try:
                        rating_elem = element.find_element(By.CSS_SELECTOR, selector)
                        rating_text = rating_elem.text.strip()
                        if rating_text:
                            rating = rating_text
                            break
                    except:
                        continue
                
                restaurant_info['rating'] = rating if rating else "N/A"
                print(f"   별점: {restaurant_info['rating']}")
                
                # 리뷰 수
                review_selectors = [
                    "a.place_review_count",
                    "span._review_count",
                    "em.place_review_count",
                    "span[class*='review']"
                ]
                
                review_count = None
                for selector in review_selectors:
                    try:
                        review_elem = element.find_element(By.CSS_SELECTOR, selector)
                        review_text = review_elem.text.strip()
                        if review_text:
                            review_count = review_text
                            break
                    except:
                        continue
                
                restaurant_info['review_count'] = review_count if review_count else "N/A"
                print(f"   리뷰 수: {restaurant_info['review_count']}")
                
                # 주소
                address_selectors = [
                    "span.place_address",
                    "span._address",
                    "div.place_address",
                    "span[class*='address']"
                ]
                
                address = None
                for selector in address_selectors:
                    try:
                        address_elem = element.find_element(By.CSS_SELECTOR, selector)
                        address = address_elem.text.strip()
                        if address:
                            break
                    except:
                        continue
                
                restaurant_info['address'] = address if address else "N/A"
                print(f"   주소: {restaurant_info['address']}")
                
                # 카테고리
                category_selectors = [
                    "span.place_category",
                    "span._category",
                    "div.place_category",
                    "span[class*='category']"
                ]
                
                category = None
                for selector in category_selectors:
                    try:
                        category_elem = element.find_element(By.CSS_SELECTOR, selector)
                        category = category_elem.text.strip()
                        if category:
                            break
                    except:
                        continue
                
                restaurant_info['category'] = category if category else "N/A"
                print(f"   카테고리: {restaurant_info['category']}")
                
                # 이미지 URL
                image_selectors = [
                    "img.place_thumb",
                    "img._thumb",
                    "div.place_thumb img",
                    "img[src*='place']",
                    "img"
                ]
                
                image_url = None
                for selector in image_selectors:
                    try:
                        img_elem = element.find_element(By.CSS_SELECTOR, selector)
                        image_url = img_elem.get_attribute('src')
                        if image_url and 'http' in image_url:
                            break
                    except:
                        continue
                
                restaurant_info['image_url'] = image_url if image_url else "N/A"
                if image_url:
                    print(f"   이미지 URL: {image_url[:50]}...")
                
                restaurants.append(restaurant_info)
                
            except Exception as e:
                print(f"음식점 {i+1} 정보 추출 중 오류: {e}")
                continue
        
        return restaurants
        
    except Exception as e:
        print(f"음식점 정보 추출 중 오류: {e}")
        return restaurants

def main():
    # 크롬 드라이버 설정
    options = webdriver.ChromeOptions()
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36')
    options.add_argument('window-size=1380,900')
    
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 15)
    
    try:
        # 검색 키워드 설정
        keyword = "신림동 짜장면"
        encoded_keyword = urllib.parse.quote(keyword)
        URL = f"https://map.naver.com/p/search/{encoded_keyword}"
        
        print(f"네이버 지도 접속 중: {URL}")
        driver.get(URL)
        time.sleep(5)
        
        # 음식점 정보 추출
        restaurants = extract_restaurant_info(driver, wait)
        
        if not restaurants:
            print("추출된 음식점 정보가 없습니다.")
            return
        
        print(f"\n총 {len(restaurants)}개의 음식점 정보를 추출했습니다.")
        
        # 결과를 JSON 파일로 저장
        output_file = f"{keyword.replace(' ', '_')}_restaurants.json"
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(restaurants, f, ensure_ascii=False, indent=2)
        
        print(f"\n결과가 {output_file}에 저장되었습니다.")
        
        # 이미지 다운로드
        image_dir = f"{keyword.replace(' ', '_')}_images"
        create_directory(image_dir)
        
        downloaded_count = 0
        for i, restaurant in enumerate(restaurants):
            if restaurant.get('image_url') and restaurant['image_url'] != "N/A":
                image_filename = f"{image_dir}/{i+1}_{restaurant['name'].replace('/', '_')}.jpg"
                if download_image(restaurant['image_url'], image_filename):
                    downloaded_count += 1
                    print(f"이미지 다운로드 완료: {restaurant['name']}")
                time.sleep(1)  # 서버 부하 방지
        
        print(f"\n총 {downloaded_count}개의 이미지가 다운로드되었습니다.")
        
        # 결과 요약 출력
        print("\n=== 크롤링 결과 요약 ===")
        for i, restaurant in enumerate(restaurants, 1):
            print(f"{i}. {restaurant['name']}")
            print(f"   별점: {restaurant['rating']} | 리뷰: {restaurant['review_count']}")
            print(f"   주소: {restaurant['address']}")
            print(f"   카테고리: {restaurant['category']}")
            print("   " + "-"*50)
        
    except Exception as e:
        print(f"오류 발생: {e}")
        
        # 디버깅을 위한 스크린샷 저장
        try:
            driver.save_screenshot("debug_screenshot.png")
            print("디버깅 스크린샷 저장: debug_screenshot.png")
        except:
            pass
    
    finally:
        input("\n엔터를 누르면 브라우저가 종료됩니다...")
        driver.quit()

if __name__ == "__main__":
    main()

네이버 지도 접속 중: https://map.naver.com/p/search/%EC%8B%A0%EB%A6%BC%EB%8F%99%20%EC%A7%9C%EC%9E%A5%EB%A9%B4
검색 결과 컨테이너를 찾을 수 없음
추출된 음식점 정보가 없습니다.

엔터를 누르면 브라우저가 종료됩니다...


In [ ]:
### 다시 정리 

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
 
from time import sleep
import random
import re
 
from selenium import webdriver
import sys
 
from selenium.webdriver.chrome.options import Options

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

service = Service(port=9999)  # 다른 포트 사용
driver = webdriver.Chrome(service=service)

##### 크롬 인터넷 설정
options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36')
options.add_argument('window-size=1380,900')
driver = webdriver.Chrome(options=options)

try:
    # 1. URL 정의
    keyword = "신림동 짜장면"
    encoded_keyword = urllib.parse.quote(keyword)
    URL = f"https://map.naver.com/p/search/{encoded_keyword}"
    
    # 2. 검색어 정리 (이미 위에서 했음)
    print(f"네이버 지도 접속 중: {URL}")
    driver.get(URL)
    time.sleep(5)
    
    # 3. 검색창 찾기 (메인 페이지에서)
    print("메인 페이지에서 검색창 찾는 중...")
    search_box = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'input_search')))
    search_box.clear()
    search_box.send_keys(keyword)
    search_box.send_keys(Keys.ENTER)
    
    print("검색 완료! 결과 대기 중...")
    time.sleep(5)
    
    # 4. iframe 접근 (검색 완료 후)
    print("iframe으로 전환 중...")
    iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe")))
    driver.switch_to.frame(iframe)
    
#     # 5. 가게정보 수집 (iframe 내에서)
#     print("가게 정보 수집 중...")
#     restaurant_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[data-id]")))
#     print(f"가게 정보 {len(restaurant_elements)}개 발견")
    
#     # 여기서 가게 정보 추출...
    
except Exception as e:
    print(f"오류 발생: {e}")

네이버 지도 접속 중: https://map.naver.com/p/search/%EC%8B%A0%EB%A6%BC%EB%8F%99%20%EC%A7%9C%EC%9E%A5%EB%A9%B4
메인 페이지에서 검색창 찾는 중...
오류 발생: name 'wait' is not defined


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
import time

# 서비스 설정
service = Service(port=9999)  # 다른 포트 사용

# 크롬 인터넷 설정
options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36')
options.add_argument('window-size=1380,900')

# 드라이버 생성 (service와 options 모두 사용)
driver = webdriver.Chrome(service=service, options=options)

# WebDriverWait 초기화 (이 부분이 빠져있었습니다!)
wait = WebDriverWait(driver, 10)

try:
    # 1. URL 정의
    keyword = "신림동 짜장면"
    encoded_keyword = urllib.parse.quote(keyword)
    URL = f"https://map.naver.com/p/search/{encoded_keyword}"
    
    # 2. 검색어 정리 (이미 위에서 했음)
    print(f"네이버 지도 접속 중: {URL}")
    driver.get(URL)
    time.sleep(5)
    
    # 3. 검색창 찾기 (메인 페이지에서)
    print("메인 페이지에서 검색창 찾는 중...")
    search_box = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'input_search')))
    search_box.clear()
    search_box.send_keys(keyword)
    search_box.send_keys(Keys.ENTER)
    
    print("검색 완료! 결과 대기 중...")
    time.sleep(5)
    
    # 4. iframe 접근 (검색 완료 후)
    print("iframe으로 전환 중...")
    iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe")))
    driver.switch_to.frame(iframe)
    
    # 5. 가게정보 수집 (iframe 내에서)
    print("가게 정보 수집 중...")
    restaurant_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[Ryr1F]")))
    print(f"가게 정보 {len(restaurant_elements)}개 발견")
    
    # 여기서 가게 정보 추출...
    
# except Exception as e:
#     print(f"오류 발생: {e}")
    
finally:
    # 드라이버 종료
    driver.quit()

네이버 지도 접속 중: https://map.naver.com/p/search/%EC%8B%A0%EB%A6%BC%EB%8F%99%20%EC%A7%9C%EC%9E%A5%EB%A9%B4
메인 페이지에서 검색창 찾는 중...
검색 완료! 결과 대기 중...
iframe으로 전환 중...
가게 정보 수집 중...


TimeoutException: Message: 


div class="Ryr1F"
id="_pcmap_list_scroll_container"

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
import time
import csv
import json

# 서비스 설정
service = Service(port=9999)
# 크롬 인터넷 설정
options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36')
options.add_argument('window-size=1380,900')
# 드라이버 생성
driver = webdriver.Chrome(service=service, options=options)
# WebDriverWait 초기화
wait = WebDriverWait(driver, 10)

# 결과 저장할 리스트
restaurant_data = []

try:
    # 1. URL 정의 및 직접 접속
    keyword = "신림동 짜장면"
    encoded_keyword = urllib.parse.quote(keyword)
    URL = f"https://map.naver.com/p/search/{encoded_keyword}"
    
    print(f"네이버 지도 접속 중: {URL}")
    driver.get(URL)
    time.sleep(5)
    
    # 2. iframe 접근
    print("iframe으로 전환 중...")
    iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe")))
    driver.switch_to.frame(iframe)
    
    # 3. 가게 리스트 수집 (li 태그들)
    print("가게 정보 수집 중...")
    restaurant_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.UEzoS")))
    print(f"가게 정보 {len(restaurant_elements)}개 발견")
    
    # 4. 각 가게 정보 추출
    for i, restaurant in enumerate(restaurant_elements):
        try:
            # 가게명 추출
            name_element = restaurant.find_element(By.CSS_SELECTOR, "span.TYaxT")
            restaurant_name = name_element.text.strip()
            
            # 가게분류 추출
            category_element = restaurant.find_element(By.CSS_SELECTOR, "span.KCMnt")
            restaurant_category = category_element.text.strip()
            
            # 데이터 저장
            restaurant_info = {
                'index': i + 1,
                'name': restaurant_name,
                'category': restaurant_category
            }
            restaurant_data.append(restaurant_info)
            
            print(f"{i+1}. {restaurant_name} - {restaurant_category}")
            
        except Exception as e:
            print(f"가게 {i+1} 정보 추출 실패: {e}")
            continue
    
    print(f"\n총 {len(restaurant_data)}개 가게 정보 수집 완료!")
    
    # 5. 파일로 저장
    # CSV 파일 저장
    csv_filename = f"{keyword}_restaurants.csv"
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['index', 'name', 'category']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(restaurant_data)
    
    # JSON 파일 저장
    json_filename = f"{keyword}_restaurants.json"
    with open(json_filename, 'w', encoding='utf-8') as jsonfile:
        json.dump(restaurant_data, jsonfile, ensure_ascii=False, indent=2)
    
    print(f"\n파일 저장 완료:")
    print(f"- CSV 파일: {csv_filename}")
    print(f"- JSON 파일: {json_filename}")
    
    # 6. 요약 정보 출력
    print(f"\n=== 수집 요약 ===")
    print(f"검색어: {keyword}")
    print(f"총 가게 수: {len(restaurant_data)}")
    print(f"가게 분류별 개수:")
    
    # 분류별 카운트
    category_count = {}
    for restaurant in restaurant_data:
        category = restaurant['category']
        category_count[category] = category_count.get(category, 0) + 1
    
    for category, count in category_count.items():
        print(f"  - {category}: {count}개")

except Exception as e:
    print(f"오류 발생: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    # 드라이버 종료
    driver.quit()

네이버 지도 접속 중: https://map.naver.com/p/search/%EC%8B%A0%EB%A6%BC%EB%8F%99%20%EC%A7%9C%EC%9E%A5%EB%A9%B4
iframe으로 전환 중...
가게 정보 수집 중...
가게 정보 10개 발견
1. 중화요리 팔공 - 중식당
2. 리옌 본점 - 중식당
3. 만 성 찬 팅 - 중식당
4. 친향중식당 - 중식당
5. 이라운지 서울대점 - 중식당
6. 윤셰프 - 중식당
7. 영신원 - 중식당
8. 거목쟁반짜장 난곡점 - 중식당
9. 강남반점 - 중식당
10. 대통자장면 - 중식당

총 10개 가게 정보 수집 완료!

파일 저장 완료:
- CSV 파일: 신림동 짜장면_restaurants.csv
- JSON 파일: 신림동 짜장면_restaurants.json

=== 수집 요약 ===
검색어: 신림동 짜장면
총 가게 수: 10
가게 분류별 개수:
  - 중식당: 10개


In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import urllib.parse
import time
import csv
import json

# 서비스 설정
service = Service(port=9999)
# 크롬 인터넷 설정
options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36')
options.add_argument('window-size=1380,900')
# 드라이버 생성
driver = webdriver.Chrome(service=service, options=options)
# WebDriverWait 초기화
wait = WebDriverWait(driver, 10)

# 결과 저장할 리스트
all_restaurant_data = []

def scroll_to_load_all_restaurants():
    """스크롤하여 모든 가게 정보 로드"""
    print("스크롤하여 모든 가게 정보 로드 중...")
    
    # 스크롤 컨테이너 찾기
    try:
        # 여러 가능한 스크롤 컨테이너 시도
        scroll_containers = [
            "div.Ryr1F",  # 주요 스크롤 컨테이너
            "div.XUrfU",  # 전체 컨테이너
            "div#_pcmap_list_scroll_container",  # ID 기반
            "ul"  # ul 태그
        ]
        
        scroll_container = None
        for selector in scroll_containers:
            try:
                scroll_container = driver.find_element(By.CSS_SELECTOR, selector)
                print(f"스크롤 컨테이너 찾음: {selector}")
                break
            except:
                continue
        
        if not scroll_container:
            print("스크롤 컨테이너를 찾을 수 없음. 페이지 전체 스크롤 사용")
            
        previous_count = 0
        max_attempts = 10
        attempt = 0
        
        while attempt < max_attempts:
            # 현재 가게 수 확인
            current_restaurants = driver.find_elements(By.CSS_SELECTOR, "li.UEzoS")
            current_count = len(current_restaurants)
            
            print(f"현재 로드된 가게 수: {current_count}")
            
            # 더 이상 로드되지 않으면 종료
            if current_count == previous_count:
                attempt += 1
                if attempt >= 3:  # 3번 연속 같으면 종료
                    print("더 이상 로드할 가게가 없음")
                    break
            else:
                attempt = 0  # 새로운 가게가 로드되면 카운터 리셋
            
            previous_count = current_count
            
            # 스크롤 실행
            if scroll_container:
                # 컨테이너 내부 스크롤
                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_container)
            else:
                # 페이지 전체 스크롤
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            time.sleep(2)  # 로딩 대기
            
        print(f"스크롤 완료. 총 {current_count}개 가게 로드됨")
        return current_count
        
    except Exception as e:
        print(f"스크롤 중 오류 발생: {e}")
        return 0

def extract_restaurant_info(page_num):
    """현재 페이지의 모든 가게 정보 추출"""
    print(f"\n=== 페이지 {page_num} 가게 정보 추출 ===")
    
    # 스크롤하여 모든 가게 로드
    total_loaded = scroll_to_load_all_restaurants()
    
    # 가게 정보 추출
    restaurant_elements = driver.find_elements(By.CSS_SELECTOR, "li.UEzoS")
    print(f"추출할 가게 수: {len(restaurant_elements)}")
    
    page_restaurants = []
    
    for i, restaurant in enumerate(restaurant_elements):
        try:
            # 가게명 추출
            name_element = restaurant.find_element(By.CSS_SELECTOR, "span.TYaxT")
            restaurant_name = name_element.text.strip()
            
            # 가게분류 추출
            category_element = restaurant.find_element(By.CSS_SELECTOR, "span.KCMnt")
            restaurant_category = category_element.text.strip()
            
            # 중복 체크 (이름 + 분류로)
            duplicate = False
            for existing in all_restaurant_data:
                if existing['name'] == restaurant_name and existing['category'] == restaurant_category:
                    duplicate = True
                    break
            
            if not duplicate:
                restaurant_info = {
                    'index': len(all_restaurant_data) + 1,
                    'page': page_num,
                    'name': restaurant_name,
                    'category': restaurant_category
                }
                page_restaurants.append(restaurant_info)
                all_restaurant_data.append(restaurant_info)
                print(f"  {len(all_restaurant_data)}. {restaurant_name} - {restaurant_category}")
            
        except Exception as e:
            print(f"가게 {i+1} 정보 추출 실패: {e}")
            continue
    
    print(f"페이지 {page_num}에서 {len(page_restaurants)}개 새로운 가게 추가")
    return len(page_restaurants)

def go_to_next_page():
    """다음 페이지로 이동"""
    try:
        # 다음 페이지 버튼 찾기
        next_button = driver.find_element(By.CSS_SELECTOR, "a.eUTV2[aria-disabled='false']")
        
        if next_button:
            print("다음 페이지 버튼 클릭")
            ActionChains(driver).move_to_element(next_button).click().perform()
            time.sleep(3)  # 페이지 로딩 대기
            return True
        else:
            print("다음 페이지 버튼이 비활성화됨")
            return False
            
    except Exception as e:
        print(f"다음 페이지 이동 실패: {e}")
        return False

try:
    # 1. URL 정의 및 직접 접속
    keyword = "신림동 짜장면"
    encoded_keyword = urllib.parse.quote(keyword)
    URL = f"https://map.naver.com/p/search/{encoded_keyword}"
    
    print(f"네이버 지도 접속 중: {URL}")
    driver.get(URL)
    time.sleep(5)
    
    # 2. iframe 접근
    print("iframe으로 전환 중...")
    iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe")))
    driver.switch_to.frame(iframe)
    
    # 3. 모든 페이지 데이터 수집
    page_num = 1
    max_pages = 10  # 최대 10페이지까지
    
    while page_num <= max_pages:
        print(f"\n========== 페이지 {page_num} ==========")
        
        # 현재 페이지 가게 정보 추출
        new_restaurants = extract_restaurant_info(page_num)
        
        if new_restaurants == 0:
            print("더 이상 새로운 가게가 없음")
            break
        
        # 다음 페이지로 이동
        if not go_to_next_page():
            print("마지막 페이지에 도달")
            break
            
        page_num += 1
    
    print(f"\n총 {len(all_restaurant_data)}개 가게 정보 수집 완료!")
    
    # 4. 파일로 저장
    if all_restaurant_data:
        # CSV 파일 저장
        csv_filename = f"{keyword}_restaurants_full.csv"
        with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['index', 'page', 'name', 'category']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(all_restaurant_data)
        
        # JSON 파일 저장
        json_filename = f"{keyword}_restaurants_full.json"
        with open(json_filename, 'w', encoding='utf-8') as jsonfile:
            json.dump(all_restaurant_data, jsonfile, ensure_ascii=False, indent=2)
        
        print(f"\n파일 저장 완료:")
        print(f"- CSV 파일: {csv_filename}")
        print(f"- JSON 파일: {json_filename}")
        
        # 5. 요약 정보 출력
        print(f"\n=== 수집 요약 ===")
        print(f"검색어: {keyword}")
        print(f"총 가게 수: {len(all_restaurant_data)}")
        print(f"수집 페이지 수: {page_num}")
        
        # 분류별 카운트
        category_count = {}
        page_count = {}
        
        for restaurant in all_restaurant_data:
            # 분류별 카운트
            category = restaurant['category']
            category_count[category] = category_count.get(category, 0) + 1
            
            # 페이지별 카운트
            page = restaurant['page']
            page_count[page] = page_count.get(page, 0) + 1
        
        print(f"\n가게 분류별 개수:")
        for category, count in sorted(category_count.items()):
            print(f"  - {category}: {count}개")
            
        print(f"\n페이지별 개수:")
        for page, count in sorted(page_count.items()):
            print(f"  - {page}페이지: {count}개")

except Exception as e:
    print(f"오류 발생: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    # 드라이버 종료
    driver.quit()

네이버 지도 접속 중: https://map.naver.com/p/search/%EC%8B%A0%EB%A6%BC%EB%8F%99%20%EC%A7%9C%EC%9E%A5%EB%A9%B4
iframe으로 전환 중...

========== 페이지 1 ==========

=== 페이지 1 가게 정보 추출 ===
스크롤하여 모든 가게 정보 로드 중...
스크롤 컨테이너 찾음: div.Ryr1F
현재 로드된 가게 수: 10
현재 로드된 가게 수: 50
현재 로드된 가게 수: 50
현재 로드된 가게 수: 50
현재 로드된 가게 수: 50
더 이상 로드할 가게가 없음
스크롤 완료. 총 50개 가게 로드됨
추출할 가게 수: 50
  1. 중화요리 팔공 - 중식당
  2. 리옌 본점 - 중식당
  3. 만 성 찬 팅 - 중식당
  4. 친향중식당 - 중식당
  5. 이라운지 서울대점 - 중식당
  6. 윤셰프 - 중식당
  7. 영신원 - 중식당
  8. 거목쟁반짜장 난곡점 - 중식당
  9. 강남반점 - 중식당
  10. 대통자장면 - 중식당
  11. 춘리마라탕 신림점 - 마라탕
  12. 미림성 - 중식당
  13. 중국성 - 중식당
  14. 동보성 - 중식당
  15. 성원우육면 - 중식당
  16. 라쿵푸마라탕 신림점 - 마라탕
  17. 참짜장기계우동 - 중식당
  18. 우짜자하우스 - 중식당
  19. 홍콩반점0410 신림고시촌점 - 중식당
  20. 신림미관 - 중식당
  21. 탑짜장 서울대점 - 중식당
  22. 소백양훠궈샤브샤브 - 중식당
  23. 신사반점 - 중식당
  24. 진 마라탕 - 중식당
  25. 대복융 - 중식당
  26. 작은중식당리우 - 중식당
  27. 만리성 - 중식당
  28. 옛날짜장 - 중식당
  29. 용궁중화요리 - 중식당
  30. 본가짜장 난곡점 - 중식당
  31. 미친마라탕 - 중식당
  32. 서울익스프레스88 신림점 - 중식당
  33. 왕짜장 신림점 - 중식당
  34. 신객잔우육면 - 중식당
  35. 화룡

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import urllib.parse
import time
import csv
import json

# 서비스 설정
service = Service(port=9999)
# 크롬 인터넷 설정
options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36')
options.add_argument('window-size=1380,900')
# 드라이버 생성
driver = webdriver.Chrome(service=service, options=options)
# WebDriverWait 초기화
wait = WebDriverWait(driver, 10)

# 결과 저장할 리스트
all_restaurant_data = []

def scroll_to_load_all_restaurants():
    """스크롤하여 모든 가게 정보 로드"""
    print("스크롤하여 모든 가게 정보 로드 중...")
    
    # 스크롤 컨테이너 찾기
    try:
        # 여러 가능한 스크롤 컨테이너 시도
        scroll_containers = [
            "div.Ryr1F",  # 주요 스크롤 컨테이너
            "div.XUrfU",  # 전체 컨테이너
            "div#_pcmap_list_scroll_container",  # ID 기반
            "ul"  # ul 태그
        ]
        
        scroll_container = None
        for selector in scroll_containers:
            try:
                scroll_container = driver.find_element(By.CSS_SELECTOR, selector)
                print(f"스크롤 컨테이너 찾음: {selector}")
                break
            except:
                continue
        
        if not scroll_container:
            print("스크롤 컨테이너를 찾을 수 없음. 페이지 전체 스크롤 사용")
            
        previous_count = 0
        max_attempts = 10
        attempt = 0
        
        while attempt < max_attempts:
            # 현재 가게 수 확인
            current_restaurants = driver.find_elements(By.CSS_SELECTOR, "li.UEzoS")
            current_count = len(current_restaurants)
            
            print(f"현재 로드된 가게 수: {current_count}")
            
            # 더 이상 로드되지 않으면 종료
            if current_count == previous_count:
                attempt += 1
                if attempt >= 3:  # 3번 연속 같으면 종료
                    print("더 이상 로드할 가게가 없음")
                    break
            else:
                attempt = 0  # 새로운 가게가 로드되면 카운터 리셋
            
            previous_count = current_count
            
            # 스크롤 실행
            if scroll_container:
                # 컨테이너 내부 스크롤
                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_container)
            else:
                # 페이지 전체 스크롤
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            time.sleep(2)  # 로딩 대기
            
        print(f"스크롤 완료. 총 {current_count}개 가게 로드됨")
        return current_count
        
    except Exception as e:
        print(f"스크롤 중 오류 발생: {e}")
        return 0

def extract_restaurant_info(page_num):
    """현재 페이지의 모든 가게 정보 추출"""
    print(f"\n=== 페이지 {page_num} 가게 정보 추출 ===")
    
    # 스크롤하여 모든 가게 로드
    total_loaded = scroll_to_load_all_restaurants()
    
    # 가게 정보 추출
    restaurant_elements = driver.find_elements(By.CSS_SELECTOR, "li.UEzoS")
    print(f"추출할 가게 수: {len(restaurant_elements)}")
    
    page_restaurants = []
    
    for i, restaurant in enumerate(restaurant_elements):
        try:
            # 가게명 추출
            name_element = restaurant.find_element(By.CSS_SELECTOR, "span.TYaxT")
            restaurant_name = name_element.text.strip()
            
            # 가게분류 추출
            category_element = restaurant.find_element(By.CSS_SELECTOR, "span.KCMnt")
            restaurant_category = category_element.text.strip()
            
            # 중복 체크 (이름 + 분류로)
            duplicate = False
            for existing in all_restaurant_data:
                if existing['name'] == restaurant_name and existing['category'] == restaurant_category:
                    duplicate = True
                    break
            
            if not duplicate:
                restaurant_info = {
                    'index': len(all_restaurant_data) + 1,
                    'page': page_num,
                    'name': restaurant_name,
                    'category': restaurant_category
                }
                page_restaurants.append(restaurant_info)
                all_restaurant_data.append(restaurant_info)
                print(f"  {len(all_restaurant_data)}. {restaurant_name} - {restaurant_category}")
            
        except Exception as e:
            print(f"가게 {i+1} 정보 추출 실패: {e}")
            continue
    
    print(f"페이지 {page_num}에서 {len(page_restaurants)}개 새로운 가게 추가")
    return len(page_restaurants)

def go_to_next_page():
    """다음 페이지로 이동"""
    try:
        # 현재 페이지 번호 확인
        try:
            current_page = driver.find_element(By.CSS_SELECTOR, "a.mBN2s.qxokY")
            print(f"현재 페이지: {current_page.text}")
        except:
            print("현재 페이지 번호 확인 실패")
        
        # 다음 페이지 버튼 찾기 (여러 방법 시도)
        next_button = None
        
        # 방법 1: aria-disabled가 false인 다음 버튼
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "a.eUTV2[aria-disabled='false']")
            print("다음 페이지 버튼 찾음 (방법 1)")
        except:
            pass
        
        # 방법 2: 단순히 eUTV2 클래스 버튼
        if not next_button:
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, "a.eUTV2")
                print("다음 페이지 버튼 찾음 (방법 2)")
            except:
                pass
        
        # 방법 3: 텍스트로 찾기
        if not next_button:
            try:
                buttons = driver.find_elements(By.CSS_SELECTOR, "div.zRM9F a")
                for button in buttons:
                    if "다음" in button.text or ">" in button.text:
                        next_button = button
                        print("다음 페이지 버튼 찾음 (방법 3)")
                        break
            except:
                pass
        
        if next_button:
            # 버튼이 비활성화되어 있는지 확인
            if next_button.get_attribute("aria-disabled") == "true":
                print("다음 페이지 버튼이 비활성화됨 (마지막 페이지)")
                return False
            
            print("다음 페이지 버튼 클릭 중...")
            
            # 버튼으로 스크롤
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            time.sleep(1)
            
            # 클릭 시도 (여러 방법)
            try:
                # 방법 1: 일반 클릭
                next_button.click()
                print("다음 페이지 버튼 클릭 성공 (방법 1)")
            except:
                try:
                    # 방법 2: JavaScript 클릭
                    driver.execute_script("arguments[0].click();", next_button)
                    print("다음 페이지 버튼 클릭 성공 (방법 2)")
                except:
                    # 방법 3: ActionChains
                    ActionChains(driver).move_to_element(next_button).click().perform()
                    print("다음 페이지 버튼 클릭 성공 (방법 3)")
            
            # 페이지 로딩 충분히 대기
            print("페이지 로딩 대기 중...")
            time.sleep(5)
            
            # 새 페이지 로드 확인
            try:
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "li.UEzoS")))
                print("새 페이지 로드 확인됨")
            except:
                print("새 페이지 로드 확인 실패")
            
            return True
        else:
            print("다음 페이지 버튼을 찾을 수 없음")
            return False
            
    except Exception as e:
        print(f"다음 페이지 이동 실패: {e}")
        import traceback
        traceback.print_exc()
        return False

try:
    # 1. URL 정의 및 직접 접속
    keyword = "신림동 짜장면"
    encoded_keyword = urllib.parse.quote(keyword)
    URL = f"https://map.naver.com/p/search/{encoded_keyword}"
    
    print(f"네이버 지도 접속 중: {URL}")
    driver.get(URL)
    time.sleep(5)
    
    # 2. iframe 접근
    print("iframe으로 전환 중...")
    iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe")))
    driver.switch_to.frame(iframe)
    
    # 3. 모든 페이지 데이터 수집
    page_num = 1
    max_pages = 10  # 최대 10페이지까지
    
    while page_num <= max_pages:
        print(f"\n========== 페이지 {page_num} ==========")
        
        # 페이지 로딩 충분히 대기
        time.sleep(3)
        
        # 현재 페이지 가게 정보 추출
        new_restaurants = extract_restaurant_info(page_num)
        
        if new_restaurants == 0:
            print("더 이상 새로운 가게가 없음")
            break
        
        # 마지막 페이지인지 미리 확인
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "a.eUTV2")
            if next_button.get_attribute("aria-disabled") == "true":
                print("마지막 페이지 도달 (버튼 비활성화)")
                break
        except:
            print("다음 페이지 버튼 확인 실패")
        
        # 다음 페이지로 이동
        print(f"\n페이지 {page_num}에서 {page_num + 1}로 이동 시도...")
        if not go_to_next_page():
            print("페이지 이동 실패 - 수집 종료")
            break
            
        page_num += 1
        
        # 무한 루프 방지
        if page_num > max_pages:
            print(f"최대 페이지 수({max_pages}) 도달")
            break
    
    print(f"\n총 {len(all_restaurant_data)}개 가게 정보 수집 완료!")
    
    # 4. 파일로 저장
    if all_restaurant_data:
        # CSV 파일 저장
        csv_filename = f"{keyword}_restaurants_full.csv"
        with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['index', 'page', 'name', 'category']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(all_restaurant_data)
        
        # JSON 파일 저장
        json_filename = f"{keyword}_restaurants_full.json"
        with open(json_filename, 'w', encoding='utf-8') as jsonfile:
            json.dump(all_restaurant_data, jsonfile, ensure_ascii=False, indent=2)
        
        print(f"\n파일 저장 완료:")
        print(f"- CSV 파일: {csv_filename}")
        print(f"- JSON 파일: {json_filename}")
        
        # 5. 요약 정보 출력
        print(f"\n=== 수집 요약 ===")
        print(f"검색어: {keyword}")
        print(f"총 가게 수: {len(all_restaurant_data)}")
        print(f"수집 페이지 수: {page_num}")
        
        # 분류별 카운트
        category_count = {}
        page_count = {}
        
        for restaurant in all_restaurant_data:
            # 분류별 카운트
            category = restaurant['category']
            category_count[category] = category_count.get(category, 0) + 1
            
            # 페이지별 카운트
            page = restaurant['page']
            page_count[page] = page_count.get(page, 0) + 1
        
        print(f"\n가게 분류별 개수:")
        for category, count in sorted(category_count.items()):
            print(f"  - {category}: {count}개")
            
        print(f"\n페이지별 개수:")
        for page, count in sorted(page_count.items()):
            print(f"  - {page}페이지: {count}개")

except Exception as e:
    print(f"오류 발생: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    # 드라이버 종료
    driver.quit()

네이버 지도 접속 중: https://map.naver.com/p/search/%EC%8B%A0%EB%A6%BC%EB%8F%99%20%EC%A7%9C%EC%9E%A5%EB%A9%B4
iframe으로 전환 중...

========== 페이지 1 ==========

=== 페이지 1 가게 정보 추출 ===
스크롤하여 모든 가게 정보 로드 중...
스크롤 컨테이너 찾음: div.Ryr1F
현재 로드된 가게 수: 10
현재 로드된 가게 수: 50
현재 로드된 가게 수: 50
현재 로드된 가게 수: 50
현재 로드된 가게 수: 50
더 이상 로드할 가게가 없음
스크롤 완료. 총 50개 가게 로드됨
추출할 가게 수: 50
  1. 중화요리 팔공 - 중식당
  2. 리옌 본점 - 중식당
  3. 만 성 찬 팅 - 중식당
  4. 친향중식당 - 중식당
  5. 이라운지 서울대점 - 중식당
  6. 윤셰프 - 중식당
  7. 영신원 - 중식당
  8. 거목쟁반짜장 난곡점 - 중식당
  9. 강남반점 - 중식당
  10. 대통자장면 - 중식당
  11. 춘리마라탕 신림점 - 마라탕
  12. 미림성 - 중식당
  13. 중국성 - 중식당
  14. 동보성 - 중식당
  15. 성원우육면 - 중식당
  16. 라쿵푸마라탕 신림점 - 마라탕
  17. 참짜장기계우동 - 중식당
  18. 우짜자하우스 - 중식당
  19. 홍콩반점0410 신림고시촌점 - 중식당
  20. 신림미관 - 중식당
  21. 탑짜장 서울대점 - 중식당
  22. 소백양훠궈샤브샤브 - 중식당
  23. 신사반점 - 중식당
  24. 진 마라탕 - 중식당
  25. 대복융 - 중식당
  26. 작은중식당리우 - 중식당
  27. 만리성 - 중식당
  28. 옛날짜장 - 중식당
  29. 용궁중화요리 - 중식당
  30. 본가짜장 난곡점 - 중식당
  31. 미친마라탕 - 중식당
  32. 서울익스프레스88 신림점 - 중식당
  33. 왕짜장 신림점 - 중식당
  34. 신객잔우육면 - 중식당
  35. 화룡

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import urllib.parse
import time
import csv
import json

# 서비스 설정
service = Service(port=9999)
# 크롬 인터넷 설정
options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36')
options.add_argument('window-size=1380,900')
# 드라이버 생성
driver = webdriver.Chrome(service=service, options=options)
# WebDriverWait 초기화
wait = WebDriverWait(driver, 10)

# 결과 저장할 리스트
all_restaurant_data = []

def scroll_to_load_all_restaurants():
    """스크롤하여 모든 가게 정보 로드 (페이지네이션 없이)"""
    print("스크롤하여 모든 가게 정보 로드 중...")
    
    # 스크롤 컨테이너 찾기
    try:
        scroll_containers = [
            "div.Ryr1F",  # 주요 스크롤 컨테이너
            "div.XUrfU",  # 전체 컨테이너
            "div#_pcmap_list_scroll_container",  # ID 기반
            "ul"  # ul 태그
        ]
        
        scroll_container = None
        for selector in scroll_containers:
            try:
                scroll_container = driver.find_element(By.CSS_SELECTOR, selector)
                print(f"스크롤 컨테이너 찾음: {selector}")
                break
            except:
                continue
        
        if not scroll_container:
            print("스크롤 컨테이너를 찾을 수 없음. 페이지 전체 스크롤 사용")
            
        previous_count = 0
        max_attempts = 20  # 더 많은 시도
        attempt = 0
        consecutive_same = 0
        
        while attempt < max_attempts:
            # 현재 가게 수 확인
            current_restaurants = driver.find_elements(By.CSS_SELECTOR, "li.UEzoS")
            current_count = len(current_restaurants)
            
            print(f"현재 로드된 가게 수: {current_count}")
            
            # 더 이상 로드되지 않으면 종료
            if current_count == previous_count:
                consecutive_same += 1
                print(f"연속 {consecutive_same}번 동일한 수")
                if consecutive_same >= 5:  # 5번 연속 같으면 종료
                    print("더 이상 로드할 가게가 없음")
                    break
            else:
                consecutive_same = 0  # 새로운 가게가 로드되면 카운터 리셋
                print(f"새로운 가게 {current_count - previous_count}개 로드됨")
            
            previous_count = current_count
            attempt += 1
            
            # 스크롤 실행 (더 큰 단위로)
            if scroll_container:
                # 컨테이너 내부 스크롤 (더 많이)
                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_container)
                time.sleep(1)
                # 한 번 더 스크롤
                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_container)
            else:
                # 페이지 전체 스크롤
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            time.sleep(2)  # 로딩 대기
            
            # 100개 이상이면 페이지네이션 체크
            if current_count >= 100:
                print("100개 이상 로드됨. 페이지네이션 확인 중...")
                break
            
        print(f"스크롤 완료. 총 {current_count}개 가게 로드됨")
        return current_count
        
    except Exception as e:
        print(f"스크롤 중 오류 발생: {e}")
        return 0

def extract_restaurant_info_simple(page_num):
    """현재 페이지의 가게 정보 추출 (스크롤 없이)"""
    print(f"\n=== 페이지 {page_num} 가게 정보 추출 ===")
    
    # 가게 정보 추출
    restaurant_elements = driver.find_elements(By.CSS_SELECTOR, "li.UEzoS")
    print(f"추출할 가게 수: {len(restaurant_elements)}")
    
    page_restaurants = []
    
    for i, restaurant in enumerate(restaurant_elements):
        try:
            # 가게명 추출
            name_element = restaurant.find_element(By.CSS_SELECTOR, "span.TYaxT")
            restaurant_name = name_element.text.strip()
            
            # 가게분류 추출
            category_element = restaurant.find_element(By.CSS_SELECTOR, "span.KCMnt")
            restaurant_category = category_element.text.strip()
            
            # 중복 체크 (이름 + 분류로)
            duplicate = False
            for existing in all_restaurant_data:
                if existing['name'] == restaurant_name and existing['category'] == restaurant_category:
                    duplicate = True
                    break
            
            if not duplicate:
                restaurant_info = {
                    'index': len(all_restaurant_data) + 1,
                    'page': page_num,
                    'name': restaurant_name,
                    'category': restaurant_category
                }
                page_restaurants.append(restaurant_info)
                all_restaurant_data.append(restaurant_info)
                print(f"  {len(all_restaurant_data)}. {restaurant_name} - {restaurant_category}")
            
        except Exception as e:
            print(f"가게 {i+1} 정보 추출 실패: {e}")
            continue
    
    print(f"페이지 {page_num}에서 {len(page_restaurants)}개 새로운 가게 추가")
    return len(page_restaurants)

def go_to_next_page():
    """다음 페이지로 이동"""
    try:
        # 현재 페이지 번호 확인
        try:
            current_page = driver.find_element(By.CSS_SELECTOR, "a.mBN2s.qxokY")
            print(f"현재 페이지: {current_page.text}")
        except:
            print("현재 페이지 번호 확인 실패")
        
        # 다음 페이지 버튼 찾기 (여러 방법 시도)
        next_button = None
        
        # 방법 1: aria-disabled가 false인 다음 버튼
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "a.eUTV2[aria-disabled='false']")
            print("다음 페이지 버튼 찾음 (방법 1)")
        except:
            pass
        
        # 방법 2: 단순히 eUTV2 클래스 버튼
        if not next_button:
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, "a.eUTV2")
                print("다음 페이지 버튼 찾음 (방법 2)")
            except:
                pass
        
        # 방법 3: 텍스트로 찾기
        if not next_button:
            try:
                buttons = driver.find_elements(By.CSS_SELECTOR, "div.zRM9F a")
                for button in buttons:
                    if "다음" in button.text or ">" in button.text:
                        next_button = button
                        print("다음 페이지 버튼 찾음 (방법 3)")
                        break
            except:
                pass
        
        if next_button:
            # 버튼이 비활성화되어 있는지 확인
            if next_button.get_attribute("aria-disabled") == "true":
                print("다음 페이지 버튼이 비활성화됨 (마지막 페이지)")
                return False
            
            print("다음 페이지 버튼 클릭 중...")
            
            # 버튼으로 스크롤
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            time.sleep(1)
            
            # 클릭 시도 (여러 방법)
            try:
                # 방법 1: 일반 클릭
                next_button.click()
                print("다음 페이지 버튼 클릭 성공 (방법 1)")
            except:
                try:
                    # 방법 2: JavaScript 클릭
                    driver.execute_script("arguments[0].click();", next_button)
                    print("다음 페이지 버튼 클릭 성공 (방법 2)")
                except:
                    # 방법 3: ActionChains
                    ActionChains(driver).move_to_element(next_button).click().perform()
                    print("다음 페이지 버튼 클릭 성공 (방법 3)")
            
            # 페이지 로딩 충분히 대기
            print("페이지 로딩 대기 중...")
            time.sleep(5)
            
            # 새 페이지 로드 확인
            try:
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "li.UEzoS")))
                print("새 페이지 로드 확인됨")
            except:
                print("새 페이지 로드 확인 실패")
            
            return True
        else:
            print("다음 페이지 버튼을 찾을 수 없음")
            return False
            
    except Exception as e:
        print(f"다음 페이지 이동 실패: {e}")
        import traceback
        traceback.print_exc()
        return False

try:
    # 1. URL 정의 및 직접 접속
    keyword = "신림동 짜장면"
    encoded_keyword = urllib.parse.quote(keyword)
    URL = f"https://map.naver.com/p/search/{encoded_keyword}"
    
    print(f"네이버 지도 접속 중: {URL}")
    driver.get(URL)
    time.sleep(5)
    
    # 2. iframe 접근
    print("iframe으로 전환 중...")
    iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe")))
    driver.switch_to.frame(iframe)
    
    # 3. 첫 번째 페이지에서 스크롤로 모든 데이터 수집
    print("========== 첫 번째 방법: 스크롤로 모든 데이터 수집 ==========")
    
    # 스크롤로 모든 가게 로드
    total_loaded = scroll_to_load_all_restaurants()
    
    # 가게 정보 추출
    restaurant_elements = driver.find_elements(By.CSS_SELECTOR, "li.UEzoS")
    print(f"추출할 가게 수: {len(restaurant_elements)}")
    
    for i, restaurant in enumerate(restaurant_elements):
        try:
            # 가게명 추출
            name_element = restaurant.find_element(By.CSS_SELECTOR, "span.TYaxT")
            restaurant_name = name_element.text.strip()
            
            # 가게분류 추출
            category_element = restaurant.find_element(By.CSS_SELECTOR, "span.KCMnt")
            restaurant_category = category_element.text.strip()
            
            restaurant_info = {
                'index': len(all_restaurant_data) + 1,
                'page': 1,
                'name': restaurant_name,
                'category': restaurant_category
            }
            all_restaurant_data.append(restaurant_info)
            print(f"  {len(all_restaurant_data)}. {restaurant_name} - {restaurant_category}")
            
        except Exception as e:
            print(f"가게 {i+1} 정보 추출 실패: {e}")
            continue
    
    # 4. 페이지네이션 시도 (스크롤로 안 되는 경우)
    if len(all_restaurant_data) < 100:  # 100개 미만이면 페이지네이션 시도
        print(f"\n========== 두 번째 방법: 페이지네이션으로 추가 데이터 수집 ==========")
        print("페이지를 새로 로드하여 페이지네이션 시도...")
        
        # 페이지 새로 로드
        driver.refresh()
        time.sleep(5)
        
        # iframe 다시 접근
        iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe")))
        driver.switch_to.frame(iframe)
        
        page_num = 1
        max_pages = 10
        
        while page_num <= max_pages:
            print(f"\n--- 페이지 {page_num} ---")
            
            # 페이지 로딩 대기
            time.sleep(3)
            
            # 현재 페이지 가게 정보 추출 (스크롤 없이)
            new_restaurants = extract_restaurant_info_simple(page_num)
            
            if new_restaurants == 0 and page_num > 1:
                print("더 이상 새로운 가게가 없음")
                break
            
            # 다음 페이지로 이동
            print(f"\n페이지 {page_num}에서 {page_num + 1}로 이동 시도...")
            if not go_to_next_page():
                print("페이지 이동 실패 - 수집 종료")
                break
                
            page_num += 1
            
            # 무한 루프 방지
            if page_num > max_pages:
                print(f"최대 페이지 수({max_pages}) 도달")
                break
    
    print(f"\n총 {len(all_restaurant_data)}개 가게 정보 수집 완료!")
    
    # 4. 파일로 저장
    if all_restaurant_data:
        # CSV 파일 저장
        csv_filename = f"{keyword}_restaurants_full.csv"
        with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['index', 'page', 'name', 'category']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(all_restaurant_data)
        
        # JSON 파일 저장
        json_filename = f"{keyword}_restaurants_full.json"
        with open(json_filename, 'w', encoding='utf-8') as jsonfile:
            json.dump(all_restaurant_data, jsonfile, ensure_ascii=False, indent=2)
        
        print(f"\n파일 저장 완료:")
        print(f"- CSV 파일: {csv_filename}")
        print(f"- JSON 파일: {json_filename}")
        
        # 5. 요약 정보 출력
        print(f"\n=== 수집 요약 ===")
        print(f"검색어: {keyword}")
        print(f"총 가게 수: {len(all_restaurant_data)}")
        print(f"총 수집 페이지 수: {max(1, page_num-1) if len(all_restaurant_data) >= 100 else 1}")
        
        # 분류별 카운트
        category_count = {}
        page_count = {}
        
        for restaurant in all_restaurant_data:
            # 분류별 카운트
            category = restaurant['category']
            category_count[category] = category_count.get(category, 0) + 1
            
            # 페이지별 카운트
            page = restaurant['page']
            page_count[page] = page_count.get(page, 0) + 1
        
        print(f"\n가게 분류별 개수:")
        for category, count in sorted(category_count.items()):
            print(f"  - {category}: {count}개")
            
        print(f"\n페이지별 개수:")
        for page, count in sorted(page_count.items()):
            print(f"  - {page}페이지: {count}개")

except Exception as e:
    print(f"오류 발생: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    # 드라이버 종료
    driver.quit()

네이버 지도 접속 중: https://map.naver.com/p/search/%EC%8B%A0%EB%A6%BC%EB%8F%99%20%EC%A7%9C%EC%9E%A5%EB%A9%B4
iframe으로 전환 중...
========== 첫 번째 방법: 스크롤로 모든 데이터 수집 ==========
스크롤하여 모든 가게 정보 로드 중...
스크롤 컨테이너 찾음: div.Ryr1F
현재 로드된 가게 수: 10
새로운 가게 10개 로드됨
현재 로드된 가게 수: 50
새로운 가게 40개 로드됨
현재 로드된 가게 수: 50
연속 1번 동일한 수
현재 로드된 가게 수: 50
연속 2번 동일한 수
현재 로드된 가게 수: 50
연속 3번 동일한 수
현재 로드된 가게 수: 50
연속 4번 동일한 수
현재 로드된 가게 수: 50
연속 5번 동일한 수
더 이상 로드할 가게가 없음
스크롤 완료. 총 50개 가게 로드됨
추출할 가게 수: 50
  1. 중화요리 팔공 - 중식당
  2. 리옌 본점 - 중식당
  3. 만 성 찬 팅 - 중식당
  4. 친향중식당 - 중식당
  5. 이라운지 서울대점 - 중식당
  6. 윤셰프 - 중식당
  7. 영신원 - 중식당
  8. 거목쟁반짜장 난곡점 - 중식당
  9. 강남반점 - 중식당
  10. 대통자장면 - 중식당
  11. 춘리마라탕 신림점 - 마라탕
  12. 미림성 - 중식당
  13. 중국성 - 중식당
  14. 동보성 - 중식당
  15. 성원우육면 - 중식당
  16. 라쿵푸마라탕 신림점 - 마라탕
  17. 참짜장기계우동 - 중식당
  18. 우짜자하우스 - 중식당
  19. 홍콩반점0410 신림고시촌점 - 중식당
  20. 신림미관 - 중식당
  21. 탑짜장 서울대점 - 중식당
  22. 소백양훠궈샤브샤브 - 중식당
  23. 신사반점 - 중식당
  24. 진 마라탕 - 중식당
  25. 대복융 - 중식당
  26. 작은중식당리우 - 중식당
  27. 만리성 - 중식당
  28. 옛날짜장 - 중식당
  29. 용궁중화요리 - 중

In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import urllib.parse
import time
import csv
import json
import requests
import os
from datetime import datetime

# 서비스 설정
service = Service(port=9999)
# 크롬 인터넷 설정
options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36')
options.add_argument('window-size=1380,900')
# 드라이버 생성
driver = webdriver.Chrome(service=service, options=options)
# WebDriverWait 초기화
wait = WebDriverWait(driver, 10)

# 결과 저장할 리스트
all_restaurant_data = []

def create_directory(keyword):
    """이미지 저장용 디렉토리 생성"""
    base_dir = f"{keyword}_restaurant_data"
    images_dir = os.path.join(base_dir, "images")
    os.makedirs(images_dir, exist_ok=True)
    return base_dir, images_dir

def download_image(url, filename, images_dir):
    """이미지 다운로드"""
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            filepath = os.path.join(images_dir, filename)
            with open(filepath, 'wb') as f:
                f.write(response.content)
            return filepath
        return None
    except Exception as e:
        print(f"이미지 다운로드 실패 ({filename}): {e}")
        return None

def get_restaurant_list():
    """왼쪽 패널에서 가게 리스트 수집"""
    print("가게 리스트 수집 중...")
    
    # 스크롤하여 모든 가게 로드
    try:
        scroll_container = driver.find_element(By.CSS_SELECTOR, "div.Ryr1F")
        print("스크롤 컨테이너 찾음")
        
        previous_count = 0
        max_attempts = 15
        
        for attempt in range(max_attempts):
            restaurant_elements = driver.find_elements(By.CSS_SELECTOR, "li.UEzoS")
            current_count = len(restaurant_elements)
            
            print(f"현재 로드된 가게 수: {current_count}")
            
            if current_count == previous_count:
                if attempt >= 3:
                    print("더 이상 로드할 가게가 없음")
                    break
            else:
                previous_count = current_count
            
            # 스크롤 실행
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_container)
            time.sleep(2)
        
        return restaurant_elements
        
    except Exception as e:
        print(f"가게 리스트 수집 실패: {e}")
        return []

def click_restaurant(restaurant_element, index):
    """가게 클릭하여 세부 정보 패널 열기"""
    try:
        # 가게 링크 찾기
        link_element = restaurant_element.find_element(By.CSS_SELECTOR, "a.place_bluelink")
        
        # 스크롤하여 요소가 보이도록 함
        driver.execute_script("arguments[0].scrollIntoView(true);", link_element)
        time.sleep(1)
        
        # 클릭
        ActionChains(driver).move_to_element(link_element).click().perform()
        print(f"가게 {index} 클릭 완료")
        
        # 세부 정보 패널 로딩 대기
        time.sleep(3)
        return True
        
    except Exception as e:
        print(f"가게 {index} 클릭 실패: {e}")
        return False

def switch_to_detail_iframe():
    """오른쪽 세부 정보 iframe으로 전환"""
    try:
        # 메인으로 돌아가기
        driver.switch_to.default_content()
        
        # 세부 정보 iframe 찾기
        detail_iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#entryIframe")))
        driver.switch_to.frame(detail_iframe)
        print("세부 정보 iframe으로 전환 완료")
        return True
        
    except Exception as e:
        print(f"세부 정보 iframe 전환 실패: {e}")
        return False

def get_home_info():
    """홈 탭에서 기본 정보 수집"""
    home_info = {}
    
    try:
        # 가게 이름
        name_element = driver.find_element(By.CSS_SELECTOR, "span.Fc1rA")
        home_info['name'] = name_element.text.strip()
        print(f"가게명: {home_info['name']}")
        
        # 가게 분류
        category_element = driver.find_element(By.CSS_SELECTOR, "span.DJJvD")
        home_info['category'] = category_element.text.strip()
        print(f"분류: {home_info['category']}")
        
        # 가게 주소
        try:
            address_element = driver.find_element(By.CSS_SELECTOR, "span.LDgIH")
            home_info['address'] = address_element.text.strip()
            print(f"주소: {home_info['address']}")
        except:
            home_info['address'] = "주소 정보 없음"
        
        # 전화번호 (있다면)
        try:
            phone_element = driver.find_element(By.CSS_SELECTOR, "span.xlx7Q")
            home_info['phone'] = phone_element.text.strip()
        except:
            home_info['phone'] = "전화번호 정보 없음"
        
        # 영업시간 (있다면)
        try:
            hours_element = driver.find_element(By.CSS_SELECTOR, "time.place_blind")
            home_info['hours'] = hours_element.text.strip()
        except:
            home_info['hours'] = "영업시간 정보 없음"
            
    except Exception as e:
        print(f"홈 정보 수집 실패: {e}")
        
    return home_info

def click_menu_tab():
    """메뉴 탭 클릭"""
    try:
        # 메뉴 탭 찾기 및 클릭
        menu_tab = driver.find_element(By.XPATH, "//a[contains(text(), '메뉴')]")
        menu_tab.click()
        time.sleep(2)
        print("메뉴 탭 클릭 완료")
        return True
    except Exception as e:
        print(f"메뉴 탭 클릭 실패: {e}")
        return False

def get_menu_info(images_dir, restaurant_name):
    """메뉴 탭에서 메뉴 정보 수집"""
    menu_list = []
    
    try:
        # 메뉴 항목들 찾기
        menu_elements = driver.find_elements(By.CSS_SELECTOR, "li.E2jtL")
        
        for i, menu_element in enumerate(menu_elements):
            menu_info = {}
            
            try:
                # 메뉴명
                name_element = menu_element.find_element(By.CSS_SELECTOR, "span.lPzHi")
                menu_info['name'] = name_element.text.strip()
                
                # 메뉴 가격
                try:
                    price_element = menu_element.find_element(By.CSS_SELECTOR, "div.GXS1X")
                    menu_info['price'] = price_element.text.strip()
                except:
                    menu_info['price'] = "가격 정보 없음"
                
                # 메뉴 이미지
                try:
                    img_element = menu_element.find_element(By.CSS_SELECTOR, "img")
                    img_url = img_element.get_attribute("src")
                    if img_url:
                        # 이미지 파일명 생성
                        img_filename = f"{restaurant_name}_menu_{i+1}_{menu_info['name']}.jpg"
                        img_filename = "".join(c for c in img_filename if c.isalnum() or c in (' ', '-', '_')).rstrip()
                        
                        # 이미지 다운로드
                        downloaded_path = download_image(img_url, img_filename, images_dir)
                        menu_info['image_path'] = downloaded_path
                    else:
                        menu_info['image_path'] = None
                except:
                    menu_info['image_path'] = None
                
                menu_list.append(menu_info)
                print(f"메뉴 {i+1}: {menu_info['name']} - {menu_info['price']}")
                
            except Exception as e:
                print(f"메뉴 {i+1} 정보 수집 실패: {e}")
                continue
                
    except Exception as e:
        print(f"메뉴 정보 수집 실패: {e}")
        
    return menu_list

def click_review_tab():
    """리뷰 탭 클릭"""
    try:
        review_tab = driver.find_element(By.XPATH, "//a[contains(text(), '리뷰')]")
        review_tab.click()
        time.sleep(2)
        print("리뷰 탭 클릭 완료")
        return True
    except Exception as e:
        print(f"리뷰 탭 클릭 실패: {e}")
        return False

def get_review_info(images_dir, restaurant_name, max_reviews=10):
    """리뷰 탭에서 리뷰 정보 수집"""
    review_list = []
    
    try:
        # 리뷰 더 보기 (있다면)
        try:
            more_button = driver.find_element(By.CSS_SELECTOR, "a.fvwqf")
            more_button.click()
            time.sleep(2)
        except:
            pass
        
        # 리뷰 항목들 찾기
        review_elements = driver.find_elements(By.CSS_SELECTOR, "li.pui__X35jYm")[:max_reviews]
        
        for i, review_element in enumerate(review_elements):
            review_info = {}
            
            try:
                # 리뷰 텍스트
                try:
                    text_element = review_element.find_element(By.CSS_SELECTOR, "span.zPfVt")
                    review_info['text'] = text_element.text.strip()
                except:
                    review_info['text'] = "리뷰 텍스트 없음"
                
                # 리뷰 날짜
                try:
                    date_element = review_element.find_element(By.CSS_SELECTOR, "time")
                    review_info['date'] = date_element.text.strip()
                except:
                    review_info['date'] = "날짜 정보 없음"
                
                # 리뷰 이미지들
                review_images = []
                try:
                    img_elements = review_element.find_elements(By.CSS_SELECTOR, "img")
                    for j, img_element in enumerate(img_elements):
                        img_url = img_element.get_attribute("src")
                        if img_url and "review" in img_url:  # 리뷰 이미지만 필터링
                            img_filename = f"{restaurant_name}_review_{i+1}_{j+1}.jpg"
                            img_filename = "".join(c for c in img_filename if c.isalnum() or c in (' ', '-', '_')).rstrip()
                            
                            downloaded_path = download_image(img_url, img_filename, images_dir)
                            if downloaded_path:
                                review_images.append(downloaded_path)
                except:
                    pass
                
                review_info['images'] = review_images
                review_list.append(review_info)
                print(f"리뷰 {i+1}: {review_info['text'][:50]}..." if len(review_info['text']) > 50 else f"리뷰 {i+1}: {review_info['text']}")
                
            except Exception as e:
                print(f"리뷰 {i+1} 정보 수집 실패: {e}")
                continue
                
    except Exception as e:
        print(f"리뷰 정보 수집 실패: {e}")
        
    return review_list

def go_back_to_list():
    """목록으로 돌아가기"""
    try:
        # 메인으로 돌아가기
        driver.switch_to.default_content()
        
        # 검색 결과 iframe으로 다시 전환
        search_iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe")))
        driver.switch_to.frame(search_iframe)
        print("목록으로 돌아가기 완료")
        return True
        
    except Exception as e:
        print(f"목록으로 돌아가기 실패: {e}")
        return False

try:
    # 1. 검색어 설정 및 접속
    keyword = "신림동 짜장면"
    encoded_keyword = urllib.parse.quote(keyword)
    URL = f"https://map.naver.com/p/search/{encoded_keyword}"
    
    print(f"네이버 지도 접속 중: {URL}")
    driver.get(URL)
    time.sleep(5)
    
    # 2. 디렉토리 생성
    base_dir, images_dir = create_directory(keyword)
    
    # 3. 검색 결과 iframe 접근
    print("검색 결과 iframe으로 전환 중...")
    search_iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe")))
    driver.switch_to.frame(search_iframe)
    
    # 4. 가게 리스트 수집
    restaurant_elements = get_restaurant_list()
    total_restaurants = len(restaurant_elements)
    print(f"총 {total_restaurants}개 가게 발견")
    
    # 5. 각 가게별로 세부 정보 수집 (처음 5개만 테스트)
    max_restaurants = min(5, total_restaurants)  # 테스트용으로 5개만
    
    for i in range(max_restaurants):
        print(f"\n========== 가게 {i+1}/{max_restaurants} ==========")
        
        try:
            # 가게 클릭
            if not click_restaurant(restaurant_elements[i], i+1):
                continue
            
            # 세부 정보 iframe으로 전환
            if not switch_to_detail_iframe():
                continue
            
            # 홈 정보 수집
            print("홈 정보 수집 중...")
            home_info = get_home_info()
            
            # 메뉴 정보 수집
            menu_info = []
            if click_menu_tab():
                print("메뉴 정보 수집 중...")
                menu_info = get_menu_info(images_dir, home_info.get('name', f'restaurant_{i+1}'))
            
            # 리뷰 정보 수집
            review_info = []
            if click_review_tab():
                print("리뷰 정보 수집 중...")
                review_info = get_review_info(images_dir, home_info.get('name', f'restaurant_{i+1}'))
            
            # 전체 정보 결합
            restaurant_data = {
                'index': i + 1,
                'home_info': home_info,
                'menu_info': menu_info,
                'review_info': review_info,
                'collected_at': datetime.now().isoformat()
            }
            
            all_restaurant_data.append(restaurant_data)
            print(f"가게 {i+1} 정보 수집 완료!")
            
            # 목록으로 돌아가기
            go_back_to_list()
            time.sleep(2)
            
        except Exception as e:
            print(f"가게 {i+1} 처리 중 오류: {e}")
            # 목록으로 돌아가기 시도
            go_back_to_list()
            continue
    
    # 6. 결과 저장
    print(f"\n총 {len(all_restaurant_data)}개 가게 상세 정보 수집 완료!")
    
    # JSON 파일 저장
    json_filename = os.path.join(base_dir, f"{keyword}_detailed_restaurants.json")
    with open(json_filename, 'w', encoding='utf-8') as jsonfile:
        json.dump(all_restaurant_data, jsonfile, ensure_ascii=False, indent=2)
    
    # CSV 파일 저장 (기본 정보만)
    csv_filename = os.path.join(base_dir, f"{keyword}_restaurants_summary.csv")
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['index', 'name', 'category', 'address', 'phone', 'menu_count', 'review_count']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for restaurant in all_restaurant_data:
            row = {
                'index': restaurant['index'],
                'name': restaurant['home_info'].get('name', ''),
                'category': restaurant['home_info'].get('category', ''),
                'address': restaurant['home_info'].get('address', ''),
                'phone': restaurant['home_info'].get('phone', ''),
                'menu_count': len(restaurant['menu_info']),
                'review_count': len(restaurant['review_info'])
            }
            writer.writerow(row)
    
    print(f"\n파일 저장 완료:")
    print(f"- 상세 정보 (JSON): {json_filename}")
    print(f"- 요약 정보 (CSV): {csv_filename}")
    print(f"- 이미지 폴더: {images_dir}")

except Exception as e:
    print(f"전체 프로세스 오류 발생: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    # 드라이버 종료
    driver.quit()

네이버 지도 접속 중: https://map.naver.com/p/search/%EC%8B%A0%EB%A6%BC%EB%8F%99%20%EC%A7%9C%EC%9E%A5%EB%A9%B4
검색 결과 iframe으로 전환 중...
가게 리스트 수집 중...
스크롤 컨테이너 찾음
현재 로드된 가게 수: 10
현재 로드된 가게 수: 50
현재 로드된 가게 수: 50
현재 로드된 가게 수: 50
더 이상 로드할 가게가 없음
총 50개 가게 발견

========== 가게 1/5 ==========
가게 1 클릭 완료
세부 정보 iframe으로 전환 완료
홈 정보 수집 중...
홈 정보 수집 실패: Message: no such element: Unable to locate element: {"method":"css selector","selector":"span.Fc1rA"}
  (Session info: chrome=138.0.7204.97); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x7ff7cf8a6f95+76917]
	GetHandleVerifier [0x0x7ff7cf8a6ff0+77008]
	(No symbol) [0x0x7ff7cf659dea]
	(No symbol) [0x0x7ff7cf6b0256]
	(No symbol) [0x0x7ff7cf6b050c]
	(No symbol) [0x0x7ff7cf703887]
	(No symbol) [0x0x7ff7cf6d84af]
	(No symbol) [0x0x7ff7cf70065c]
	(No symbol) [0x0x7ff7cf6d8243]
	(No symbol) [0x0x7ff7cf6a1431]
	(No symbol) [0x0x7ff7cf6a

리뷰 탭 클릭 완료
리뷰 정보 수집 중...
가게 4 정보 수집 완료!
목록으로 돌아가기 완료

========== 가게 5/5 ==========
가게 5 클릭 완료
세부 정보 iframe으로 전환 완료
홈 정보 수집 중...
홈 정보 수집 실패: Message: no such element: Unable to locate element: {"method":"css selector","selector":"span.Fc1rA"}
  (Session info: chrome=138.0.7204.97); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x7ff7cf8a6f95+76917]
	GetHandleVerifier [0x0x7ff7cf8a6ff0+77008]
	(No symbol) [0x0x7ff7cf659dea]
	(No symbol) [0x0x7ff7cf6b0256]
	(No symbol) [0x0x7ff7cf6b050c]
	(No symbol) [0x0x7ff7cf703887]
	(No symbol) [0x0x7ff7cf6d84af]
	(No symbol) [0x0x7ff7cf70065c]
	(No symbol) [0x0x7ff7cf6d8243]
	(No symbol) [0x0x7ff7cf6a1431]
	(No symbol) [0x0x7ff7cf6a21c3]
	GetHandleVerifier [0x0x7ff7cfb7d2cd+3051437]
	GetHandleVerifier [0x0x7ff7cfb77923+3028483]
	GetHandleVerifier [0x0x7ff7cfb958bd+3151261]
	GetHandleVerifier [0x0x7ff7cf8c185e+185662]
	G

In [ ]:
# 개선된 네이버 지도 크롤링 코드

import os
import time
import json
import csv
import requests
import urllib.parse
import re
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service

# 서비스 설정
service = Service(port=9999)
# 크롬 인터넷 설정
options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36')
options.add_argument('window-size=1380,900')
# 드라이버 생성
driver = webdriver.Chrome(service=service, options=options)
# WebDriverWait 초기화
wait = WebDriverWait(driver, 10)

# 결과 저장할 리스트
all_restaurant_data = []

def create_directory(keyword):
    """이미지 저장용 디렉토리 생성"""
    base_dir = f"{keyword}_restaurant_data"
    images_dir = os.path.join(base_dir, "images")
    os.makedirs(images_dir, exist_ok=True)
    return base_dir, images_dir

def download_image(url, filename, images_dir):
    """이미지 다운로드"""
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            filepath = os.path.join(images_dir, filename)
            with open(filepath, 'wb') as f:
                f.write(response.content)
            return filepath
        return None
    except Exception as e:
        print(f"이미지 다운로드 실패 ({filename}): {e}")
        return None

def get_restaurant_list():
    """왼쪽 패널에서 가게 리스트 수집"""
    print("가게 리스트 수집 중...")
    
    # 스크롤하여 모든 가게 로드
    try:
        scroll_container = driver.find_element(By.CSS_SELECTOR, "div.Ryr1F")
        print("스크롤 컨테이너 찾음")
        
        previous_count = 0
        max_attempts = 15
        
        for attempt in range(max_attempts):
            restaurant_elements = driver.find_elements(By.CSS_SELECTOR, "li.UEzoS")
            current_count = len(restaurant_elements)
            
            print(f"현재 로드된 가게 수: {current_count}")
            
            if current_count == previous_count:
                if attempt >= 3:
                    print("더 이상 로드할 가게가 없음")
                    break
            else:
                previous_count = current_count
            
            # 스크롤 실행
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_container)
            time.sleep(2)
        
        return restaurant_elements
        
    except Exception as e:
        print(f"가게 리스트 수집 실패: {e}")
        return []

def click_restaurant(restaurant_element, index):
    """가게 클릭하여 세부 정보 패널 열기"""
    try:
        # 가게 링크 찾기
        link_element = restaurant_element.find_element(By.CSS_SELECTOR, "a.place_bluelink")
        
        # 스크롤하여 요소가 보이도록 함
        driver.execute_script("arguments[0].scrollIntoView(true);", link_element)
        time.sleep(1)
        
        # 클릭
        ActionChains(driver).move_to_element(link_element).click().perform()
        print(f"가게 {index} 클릭 완료")
        
        # 세부 정보 패널 로딩 대기
        time.sleep(3)
        return True
        
    except Exception as e:
        print(f"가게 {index} 클릭 실패: {e}")
        return False

def switch_to_detail_iframe():
    """오른쪽 세부 정보 iframe으로 전환"""
    try:
        # 메인으로 돌아가기
        driver.switch_to.default_content()
        
        # 세부 정보 iframe 찾기
        detail_iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#entryIframe")))
        driver.switch_to.frame(detail_iframe)
        print("세부 정보 iframe으로 전환 완료")
        return True
        
    except Exception as e:
        print(f"세부 정보 iframe 전환 실패: {e}")
        return False

def get_home_info():
    """홈 탭에서 기본 정보 수집"""
    home_info = {}
    
    try:
        # 가게 이름
        name_element = driver.find_element(By.CSS_SELECTOR, "span.GHAhO")
        home_info['name'] = name_element.text.strip()
        print(f"가게명: {home_info['name']}")
        
        # 가게 분류
        category_element = driver.find_element(By.CSS_SELECTOR, "span.lnJFt")
        home_info['category'] = category_element.text.strip()
        print(f"분류: {home_info['category']}")
        
        # 가게 주소
        try:
            address_element = driver.find_element(By.CSS_SELECTOR, "span.LDgIH")
            home_info['address'] = address_element.text.strip()
            print(f"주소: {home_info['address']}")
        except:
            home_info['address'] = "주소 정보 없음"
        
        # 전화번호 (있다면)
        try:
            phone_element = driver.find_element(By.CSS_SELECTOR, "span.xlx7Q")
            home_info['phone'] = phone_element.text.strip()
        except:
            home_info['phone'] = "전화번호 정보 없음"
        
        # 영업시간 (있다면)
        try:
            hours_element = driver.find_element(By.CSS_SELECTOR, "time.H3ua4")
            home_info['hours'] = hours_element.text.strip()
        except:
            home_info['hours'] = "영업시간 정보 없음"
            
    except Exception as e:
        print(f"홈 정보 수집 실패: {e}")
        
    return home_info

def click_menu_tab():
    """메뉴 탭 클릭"""
    try:
        # 메뉴 탭 찾기 및 클릭 (수정된 셀렉터)
        menu_tab = driver.find_element(By.CSS_SELECTOR, "a._tab-menu[data-index='1']")
        menu_tab.click()
        time.sleep(2)
        print("메뉴 탭 클릭 완료")
        return True
    except Exception as e:
        try:
            # 대안 셀렉터
            menu_tab = driver.find_element(By.XPATH, "//span[contains(text(), '메뉴')]/parent::a")
            menu_tab.click()
            time.sleep(2)
            print("메뉴 탭 클릭 완료 (대안 방법)")
            return True
        except Exception as e2:
            print(f"메뉴 탭 클릭 실패: {e}, {e2}")
            return False

def click_more_menu_button():
    """메뉴 더보기 버튼 클릭"""
    try:
        # 여러 더보기 버튼 셀렉터 시도
        more_button_selectors = [
            "a.fvwqf",
            "button.fvwqf",
            "a[class*='more']",
            "button[class*='more']",
            "//a[contains(text(), '더보기')]",
            "//button[contains(text(), '더보기')]"
        ]
        
        for selector in more_button_selectors:
            try:
                if selector.startswith("//"):
                    more_button = driver.find_element(By.XPATH, selector)
                else:
                    more_button = driver.find_element(By.CSS_SELECTOR, selector)
                
                if more_button.is_displayed() and more_button.is_enabled():
                    # 요소가 보이도록 스크롤
                    driver.execute_script("arguments[0].scrollIntoView(true);", more_button)
                    time.sleep(1)
                    
                    # JavaScript로 클릭
                    driver.execute_script("arguments[0].click();", more_button)
                    time.sleep(2)
                    print("메뉴 더보기 버튼 클릭 완료")
                    return True
            except:
                continue
                
        print("메뉴 더보기 버튼이 없거나 이미 모든 메뉴가 표시됨")
        return False
        
    except Exception as e:
        print(f"메뉴 더보기 버튼 처리 실패: {e}")
        return False

def get_menu_photos(images_dir, restaurant_name):
    """메뉴판 이미지 다운로드"""
    menu_photos = []
    
    try:
        # 메뉴판 이미지 섹션 찾기 (여러 셀렉터 시도)
        section_selectors = [
            "div.place_section",
            "div[class*='photo']",
            "div[class*='image']",
            "div[class*='menu']"
        ]
        
        menu_photo_section = None
        for selector in section_selectors:
            try:
                menu_photo_section = driver.find_element(By.CSS_SELECTOR, selector)
                break
            except:
                continue
        
        if not menu_photo_section:
            print("메뉴판 이미지 섹션을 찾을 수 없습니다")
            return menu_photos
        
        # 이미지 요소들 찾기 (여러 셀렉터 시도)
        img_selectors = ["img.K0PDV", "img", "img[alt*='메뉴']"]
        img_elements = []
        
        for img_selector in img_selectors:
            try:
                elements = menu_photo_section.find_elements(By.CSS_SELECTOR, img_selector)
                if elements:
                    img_elements = elements
                    break
            except:
                continue
        
        for i, img_element in enumerate(img_elements):
            try:
                img_url = img_element.get_attribute("src")
                alt_text = img_element.get_attribute("alt", "").lower()
                
                # 메뉴판 이미지인지 확인
                if img_url and ("menu" in alt_text or "메뉴" in alt_text or len(alt_text) == 0):
                    # 메뉴판 이미지 파일명 생성
                    img_filename = f"{restaurant_name}_menu_photo_{i+1}.jpg"
                    img_filename = "".join(c for c in img_filename if c.isalnum() or c in (' ', '-', '_')).rstrip()
                    
                    # 이미지 다운로드
                    downloaded_path = download_image(img_url, img_filename, images_dir)
                    if downloaded_path:
                        menu_photos.append(downloaded_path)
                        print(f"메뉴판 이미지 {i+1} 다운로드 완료")
            except Exception as e:
                print(f"메뉴판 이미지 {i+1} 다운로드 실패: {e}")
                continue
                
        # 메뉴판 이미지 슬라이드 버튼 클릭해서 더 많은 이미지 로드
        try:
            # 여러 슬라이드 버튼 셀렉터 시도
            slide_selectors = ["a.ZCqf_", "button.ZCqf_", "[class*='next']", "[class*='slide']"]
            
            for slide_selector in slide_selectors:
                try:
                    next_button = menu_photo_section.find_element(By.CSS_SELECTOR, slide_selector)
                    if next_button.is_displayed() and next_button.is_enabled():
                        click_count = 0
                        max_clicks = 5  # 최대 5번 클릭
                        
                        while click_count < max_clicks:
                            try:
                                # JavaScript로 클릭
                                driver.execute_script("arguments[0].click();", next_button)
                                time.sleep(1)
                                click_count += 1
                                
                                # 새로운 이미지들 찾기
                                new_img_elements = menu_photo_section.find_elements(By.CSS_SELECTOR, img_selector)
                                for j, new_img_element in enumerate(new_img_elements[len(img_elements):]):
                                    try:
                                        img_url = new_img_element.get_attribute("src")
                                        if img_url:
                                            img_filename = f"{restaurant_name}_menu_photo_{len(img_elements)+j+1}.jpg"
                                            img_filename = "".join(c for c in img_filename if c.isalnum() or c in (' ', '-', '_')).rstrip()
                                            
                                            downloaded_path = download_image(img_url, img_filename, images_dir)
                                            if downloaded_path:
                                                menu_photos.append(downloaded_path)
                                    except:
                                        continue
                            except:
                                break
                        break
                except:
                    continue
                    
        except Exception as e:
            print(f"메뉴판 이미지 슬라이드 처리 실패: {e}")
            
    except Exception as e:
        print(f"메뉴판 이미지 수집 실패: {e}")
    
    return menu_photos

def get_menu_info(images_dir, restaurant_name):
    """메뉴 탭에서 메뉴 정보 수집"""
    menu_list = []
    
    try:
        # 메뉴 더보기 버튼 클릭
        click_more_menu_button()
        
        # 메뉴 항목들 찾기 (수정된 셀렉터)
        menu_elements = driver.find_elements(By.CSS_SELECTOR, "li.E2jtL")
        
        for i, menu_element in enumerate(menu_elements):
            menu_info = {}
            
            try:
                # 메뉴명 (수정된 셀렉터)
                name_element = menu_element.find_element(By.CSS_SELECTOR, "span.lPzHi")
                menu_info['name'] = name_element.text.strip()
                
                # 메뉴 가격 (수정된 셀렉터)
                try:
                    price_element = menu_element.find_element(By.CSS_SELECTOR, "div.GXS1X em")
                    menu_info['price'] = price_element.text.strip()
                except:
                    menu_info['price'] = "가격 정보 없음"
                
                # 메뉴 설명 (수정된 셀렉터)
                try:
                    desc_element = menu_element.find_element(By.CSS_SELECTOR, "div.TRxGt")
                    menu_info['description'] = desc_element.text.strip()
                except:
                    menu_info['description'] = "설명 없음"
                
                # 메뉴 이미지 (수정된 방식)
                try:
                    # 여러 이미지 셀렉터 시도
                    img_element = None
                    selectors = ["img.K0PDV", "img", "div.xPf1B img"]
                    
                    for selector in selectors:
                        try:
                            img_element = menu_element.find_element(By.CSS_SELECTOR, selector)
                            break
                        except:
                            continue
                    
                    if img_element:
                        img_url = img_element.get_attribute("src")
                        if img_url:
                            # 이미지 파일명 생성
                            img_filename = f"{restaurant_name}_menu_item_{i+1}_{menu_info['name']}.jpg"
                            img_filename = "".join(c for c in img_filename if c.isalnum() or c in (' ', '-', '_')).rstrip()
                            
                            # 이미지 다운로드
                            downloaded_path = download_image(img_url, img_filename, images_dir)
                            menu_info['image_path'] = downloaded_path
                        else:
                            menu_info['image_path'] = None
                    else:
                        menu_info['image_path'] = None
                except:
                    menu_info['image_path'] = None
                
                menu_list.append(menu_info)
                print(f"메뉴 {i+1}: {menu_info['name']} - {menu_info['price']}")
                
            except Exception as e:
                print(f"메뉴 {i+1} 정보 수집 실패: {e}")
                continue
                
    except Exception as e:
        print(f"메뉴 정보 수집 실패: {e}")
    
    # 메뉴판 사진도 수집
    menu_photos = get_menu_photos(images_dir, restaurant_name)
    
    return menu_list, menu_photos

def click_review_tab():
    """리뷰 탭 클릭"""
    try:
        # 페이지 스크롤하여 탭이 보이도록 함
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(1)
        
        # 리뷰 탭 찾기 및 클릭 (여러 방법 시도)
        review_selectors = [
            "//a[contains(text(), '리뷰')]",
            "//a[contains(@href, 'review')]",
            "a[href*='review']",
            "a._tab-menu[data-index='2']"
        ]
        
        for selector in review_selectors:
            try:
                if selector.startswith("//"):
                    review_tab = driver.find_element(By.XPATH, selector)
                else:
                    review_tab = driver.find_element(By.CSS_SELECTOR, selector)
                
                # 요소가 보이도록 스크롤
                driver.execute_script("arguments[0].scrollIntoView(true);", review_tab)
                time.sleep(1)
                
                # JavaScript로 클릭 시도 (element click intercepted 해결)
                driver.execute_script("arguments[0].click();", review_tab)
                time.sleep(2)
                print("리뷰 탭 클릭 완료")
                return True
                
            except Exception as e:
                continue
        
        print("리뷰 탭을 찾을 수 없습니다")
        return False
        
    except Exception as e:
        print(f"리뷰 탭 클릭 실패: {e}")
        return False

def click_more_review_button():
    """리뷰 더보기 버튼 클릭"""
    try:
        # 여러 더보기 버튼 셀렉터 시도
        more_button_selectors = [
            "a.fvwqf",
            "button.fvwqf", 
            "a[class*='more']",
            "button[class*='more']",
            "//a[contains(text(), '더보기')]",
            "//button[contains(text(), '더보기')]",
            "//a[contains(text(), '리뷰 더보기')]"
        ]
        
        for selector in more_button_selectors:
            try:
                if selector.startswith("//"):
                    more_button = driver.find_element(By.XPATH, selector)
                else:
                    more_button = driver.find_element(By.CSS_SELECTOR, selector)
                
                if more_button.is_displayed() and more_button.is_enabled():
                    # 요소가 보이도록 스크롤
                    driver.execute_script("arguments[0].scrollIntoView(true);", more_button)
                    time.sleep(1)
                    
                    # JavaScript로 클릭
                    driver.execute_script("arguments[0].click();", more_button)
                    time.sleep(2)
                    print("리뷰 더보기 버튼 클릭 완료")
                    return True
            except:
                continue
                
        print("리뷰 더보기 버튼이 없거나 이미 모든 리뷰가 표시됨")
        return False
        
    except Exception as e:
        print(f"리뷰 더보기 버튼 처리 실패: {e}")
        return False

def get_review_info(images_dir, restaurant_name, max_reviews=20):
    """리뷰 탭에서 리뷰 정보 수집"""
    review_list = []
    
    try:
        # 리뷰 더보기 버튼 클릭
        click_more_review_button()
        
        # 리뷰 항목들 찾기 (수정된 셀렉터)
        review_elements = driver.find_elements(By.CSS_SELECTOR, "li.place_apply_pui.EjjAW")[:max_reviews]
        
        for i, review_element in enumerate(review_elements):
            review_info = {}
            
            try:
                # 리뷰 텍스트
                try:
                    text_element = review_element.find_element(By.CSS_SELECTOR, "span.zPfVt")
                    review_info['text'] = text_element.text.strip()
                except:
                    review_info['text'] = "리뷰 텍스트 없음"
                
                # 리뷰 날짜 (여러 패턴 시도)
                try:
                    # 첫 번째 시도: time 태그
                    date_element = review_element.find_element(By.CSS_SELECTOR, "time")
                    review_info['date'] = date_element.text.strip()
                except:
                    try:
                        # 두 번째 시도: span.pui__gfuUIT 내부
                        date_element = review_element.find_element(By.CSS_SELECTOR, "span.pui__gfuUIT")
                        review_info['date'] = date_element.text.strip()
                    except:
                        try:
                            # 세 번째 시도: div.pui__QKE5Pr 내부
                            date_element = review_element.find_element(By.CSS_SELECTOR, "div.pui__QKE5Pr")
                            # blind 텍스트 제외하고 날짜만 추출
                            date_text = date_element.text.strip()
                            # "방문" 또는 기타 텍스트 제거하고 날짜만 추출
                            import re
                            date_match = re.search(r'(\d{4}\.\d{1,2}\.\d{1,2}|\d{1,2}\.\d{1,2}|\d+일전|\d+주전|\d+개월전|\d+년전)', date_text)
                            if date_match:
                                review_info['date'] = date_match.group(1)
                            else:
                                review_info['date'] = date_text
                        except:
                            review_info['date'] = "날짜 정보 없음"
                
                # 리뷰어 정보 수집
                reviewer_info = {}
                try:
                    # 리뷰어 이름
                    reviewer_name_element = review_element.find_element(By.CSS_SELECTOR, "span.pui__gfuUIT")
                    reviewer_info['name'] = reviewer_name_element.text.strip()
                except:
                    reviewer_info['name'] = "리뷰어 정보 없음"
                
                try:
                    # 리뷰어 방문 정보 (재방문, 첫 방문 등)
                    visit_info_element = review_element.find_element(By.CSS_SELECTOR, "span.pui__blind")
                    visit_info = visit_info_element.text.strip()
                    if visit_info:
                        reviewer_info['visit_type'] = visit_info
                    else:
                        reviewer_info['visit_type'] = "방문 정보 없음"
                except:
                    reviewer_info['visit_type'] = "방문 정보 없음"
                review_info['reviewer'] = reviewer_info
                
                # 리뷰 평점 (별점이 있다면)
                try:
                    rating_element = review_element.find_element(By.CSS_SELECTOR, "span.pui__rating, div.pui__rating")
                    rating_text = rating_element.get_attribute("aria-label") or rating_element.text
                    # 별점 숫자 추출
                    rating_match = re.search(r'(\d+(?:\.\d+)?)', rating_text)
                    if rating_match:
                        review_info['rating'] = rating_match.group(1)
                                        
                # 리뷰 이미지들
                except:
                    review_info['rating'] = "평점 정보 없음"
                review_images = []
                try:
                    # 리뷰 이미지 슬라이드 처리
                    img_elements = review_element.find_elements(By.CSS_SELECTOR, "img")
                    for j, img_element in enumerate(img_elements):
                        img_url = img_element.get_attribute("src")
                        if img_url and ("review" in img_url or "ldb-phinf" in img_url):  # 리뷰 이미지 필터링
                            img_filename = f"{restaurant_name}_review_{i+1}_{j+1}.jpg"
                            img_filename = "".join(c for c in img_filename if c.isalnum() or c in (' ', '-', '_')).rstrip()
                            
                            downloaded_path = download_image(img_url, img_filename, images_dir)
                            if downloaded_path:
                                review_images.append(downloaded_path)
                    
                    # 리뷰 이미지 슬라이드 버튼 클릭해서 더 많은 이미지 보기
                    try:
                        next_button = review_element.find_element(By.CSS_SELECTOR, "a.ZCqf_")
                        click_count = 0
                        max_clicks = 5  # 리뷰당 최대 5번 클릭
                        
                        while click_count < max_clicks:
                            if next_button.is_displayed() and next_button.is_enabled():
                                next_button.click()
                                time.sleep(1)
                                click_count += 1
                                
                                # 새로운 이미지들 찾기
                                new_img_elements = review_element.find_elements(By.CSS_SELECTOR, "img")
                                for k, new_img_element in enumerate(new_img_elements[len(img_elements):]):
                                    try:
                                        img_url = new_img_element.get_attribute("src")
                                        if img_url and ("review" in img_url or "ldb-phinf" in img_url):
                                            img_filename = f"{restaurant_name}_review_{i+1}_{len(img_elements)+k+1}.jpg"
                                            img_filename = "".join(c for c in img_filename if c.isalnum() or c in (' ', '-', '_')).rstrip()
                                            
                                            downloaded_path = download_image(img_url, img_filename, images_dir)
                                            if downloaded_path:
                                                review_images.append(downloaded_path)
                                    except:
                                        continue
                            else:
                                break
                    except:
                        pass
                        
                except Exception as e:
                    print(f"리뷰 {i+1} 이미지 수집 실패: {e}")
                
                review_info['images'] = review_images
                review_list.append(review_info)
                print(f"리뷰 {i+1}: {review_info['text'][:50]}..." if len(review_info['text']) > 50 else f"리뷰 {i+1}: {review_info['text']}")
                
            except Exception as e:
                print(f"리뷰 {i+1} 정보 수집 실패: {e}")
                continue
                
    except Exception as e:
        print(f"리뷰 정보 수집 실패: {e}")
        
    return review_list

def go_back_to_list():
    """목록으로 돌아가기"""
    try:
        # 메인으로 돌아가기
        driver.switch_to.default_content()
        
        # 검색 결과 iframe으로 다시 전환
        search_iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe")))
        driver.switch_to.frame(search_iframe)
        print("목록으로 돌아가기 완료")
        return True
        
    except Exception as e:
        print(f"목록으로 돌아가기 실패: {e}")
        return False

try:
    # 1. 검색어 설정 및 접속
    keyword = "신림동 짜장면"
    encoded_keyword = urllib.parse.quote(keyword)
    URL = f"https://map.naver.com/p/search/{encoded_keyword}"
    
    print(f"네이버 지도 접속 중: {URL}")
    driver.get(URL)
    time.sleep(5)
    
    # 2. 디렉토리 생성
    base_dir, images_dir = create_directory(keyword)
    
    # 3. 검색 결과 iframe 접근
    print("검색 결과 iframe으로 전환 중...")
    search_iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe")))
    driver.switch_to.frame(search_iframe)
    
    # 4. 가게 리스트 수집
    restaurant_elements = get_restaurant_list()
    total_restaurants = len(restaurant_elements)
    print(f"총 {total_restaurants}개 가게 발견")
    
    # 5. 각 가게별로 세부 정보 수집 (처음 5개만 테스트)
    max_restaurants = min(5, total_restaurants)  # 테스트용으로 5개만
    
    for i in range(max_restaurants):
        print(f"\n========== 가게 {i+1}/{max_restaurants} ==========")
        
        try:
            # 가게 클릭
            if not click_restaurant(restaurant_elements[i], i+1):
                continue
            
            # 세부 정보 iframe으로 전환
            if not switch_to_detail_iframe():
                continue
            
            # 홈 정보 수집
            print("홈 정보 수집 중...")
            home_info = get_home_info()
            
            # 메뉴 정보 수집
            menu_info = []
            menu_photos = []
            if click_menu_tab():
                print("메뉴 정보 수집 중...")
                menu_info, menu_photos = get_menu_info(images_dir, home_info.get('name', f'restaurant_{i+1}'))
            
            # 리뷰 정보 수집
            review_info = []
            if click_review_tab():
                print("리뷰 정보 수집 중...")
                review_info = get_review_info(images_dir, home_info.get('name', f'restaurant_{i+1}'))
            
            # 전체 정보 결합
            restaurant_data = {
                'index': i + 1,
                'home_info': home_info,
                'menu_info': menu_info,
                'menu_photos': menu_photos,
                'review_info': review_info,
                'collected_at': datetime.now().isoformat()
            }
            
            all_restaurant_data.append(restaurant_data)
            print(f"가게 {i+1} 정보 수집 완료!")
            
            # 목록으로 돌아가기
            go_back_to_list()
            time.sleep(2)
            
        except Exception as e:
            print(f"가게 {i+1} 처리 중 오류: {e}")
            # 목록으로 돌아가기 시도
            go_back_to_list()
            continue
    
    # 6. 결과 저장
    print(f"\n총 {len(all_restaurant_data)}개 가게 상세 정보 수집 완료!")
    
    # JSON 파일 저장
    json_filename = os.path.join(base_dir, f"{keyword}_detailed_restaurants.json")
    with open(json_filename, 'w', encoding='utf-8') as jsonfile:
        json.dump(all_restaurant_data, jsonfile, ensure_ascii=False, indent=2)
    
    # CSV 파일 저장 (기본 정보만)
    csv_filename = os.path.join(base_dir, f"{keyword}_restaurants_summary.csv")
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['index', 'name', 'category', 'address', 'phone', 'menu_count', 'menu_photos_count', 'review_count']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for restaurant in all_restaurant_data:
            row = {
                'index': restaurant['index'],
                'name': restaurant['home_info'].get('name', ''),
                'category': restaurant['home_info'].get('category', ''),
                'address': restaurant['home_info'].get('address', ''),
                'phone': restaurant['home_info'].get('phone', ''),
                'menu_count': len(restaurant['menu_info']),
                'menu_photos_count': len(restaurant['menu_photos']),
                'review_count': len(restaurant['review_info'])
            }
            writer.writerow(row)
    
    print(f"\n파일 저장 완료:")
    print(f"- 상세 정보 (JSON): {json_filename}")
    print(f"- 요약 정보 (CSV): {csv_filename}")
    print(f"- 이미지 폴더: {images_dir}")

except Exception as e:
    print(f"전체 프로세스 오류 발생: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    # 드라이버 종료
    driver.quit()